# 🏥 Guía Hackathon Salud NHANES - Coach de Bienestar Preventivo
## Duoc UC 2025 - 27 horas de desarrollo

**Objetivo:** Sistema híbrido ML + LLM para predicción de riesgo cardiometabólico y coaching personalizado

### 📋 Checklist de Entregables
- [ ] Modelo ML con AUROC ≥ 0.80
- [ ] API FastAPI con /predict y /coach
- [ ] App en Streamlit/Gradio deployada en HF Spaces
- [ ] Validación temporal sin fuga de datos
- [ ] Métricas de fairness por subgrupos
- [ ] RAG con citas a /kb local
- [ ] PDF descargable del plan
- [ ] Presentación de 10 min

---

## 🚀 FASE 0: Setup Inicial (30 min - H0 a H0.5)

### Instalación de dependencias

In [17]:
# Instalar dependencias desde requirements.txt
# Esto asegura que todas las librerías estén en las versiones correctas

!pip install -r requirements.txt

# Si prefieres instalar individualmente (no recomendado):
# !pip install -q pandas numpy scikit-learn xgboost lightgbm \
#     fastapi uvicorn pydantic streamlit gradio \
#     openai shap matplotlib seaborn plotly \
#     reportlab fpdf rank-bm25 python-multipart joblib requests


  Using cached pandas-2.0.3.tar.gz (5.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached numpy-1.24.3.tar.gz (10.9 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
ERROR: Exception:
Traceback (most recent call last):
  File "/Users/v1tso/dev/new-hack/.venv/lib/python3.13/site-packages/pip/_internal/cli/base_command.py", line 107, in _run_wrapper
    status = _inner_run()
  File "/Users/v1tso/dev/new-hack/.venv/lib/python3.13/site-packages/pip/_internal/cli/base_command.py", line 98, in _inner_run
    return self.run(options, args)
           ~~~~~~~~^^^^^^^^^^^^^^^
  File "/Users/v1tso/dev/new-hack/.venv/lib/python3.13/site-packages/pip/_internal/cli/req_command.py", line 85, in wrapper
    return func(self, options, args)
  File "/Users/v1tso/dev/new-hack/.venv/lib/python3.13/site-packages/pip/_internal/commands/install.py", line 388, in r

In [ ]:
# Imports generales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
import json
from datetime import datetime

warnings.filterwarnings('ignore')
np.random.seed(42)

# Configuración visual
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Setup completado")

ModuleNotFoundError: No module named 'pandas'

: 

: 

### 📂 Estructura de Datos Esperada

**IMPORTANTE:** Antes de ejecutar el código, asegúrate de tener los datos NHANES en formato CSV.

**Estructura de directorios esperada:**
```
./data/
├── DEMO_2015_2016.csv    # Demographics (OBLIGATORIO)
├── EXAM_2015_2016.csv    # Examination (recomendado)
├── LAB_2015_2016.csv     # Laboratory (recomendado para labels)
├── QUEST_2015_2016.csv   # Questionnaire (opcional)
├── DIET_2015_2016.csv    # Dietary (opcional)
├── DEMO_2017_2018.csv
└── ... (y así para cada ciclo permitido)
```

**Formato de nombres:**
- `DEMO_{CICLO}.csv` donde CICLO = `2015_2016` (con guión bajo)
- `EXAM_{CICLO}.csv`
- `LAB_{CICLO}.csv`
- `QUEST_{CICLO}.csv`
- `DIET_{CICLO}.csv`

**Ciclos soportados:**
- Entrenamiento: `2015-2016`
- Test: `2017-2018`

**Columna obligatoria:**
- Todos los archivos deben tener la columna `SEQN` (ID único del participante)

Si no tienes los datos, el código mostrará mensajes de advertencia pero no fallará.


### ⚠️ IMPORTANTE: Los Datos NHANES NO Vienen en CSV

**Los datos de NHANES se distribuyen en formato SAS Transport File (.XPT) y necesitan ser convertidos a CSV.**

**Pasos para obtener los datos:**

1. **Descargar archivos .XPT:**

   **Opción A - Descarga Automática (Intentar primero):**
   ```bash
   # Descargar un módulo de prueba
   python descargar_nhanes.py --cycle 2017-2018 --module DEMO
   
   # Descargar múltiples módulos
   python descargar_nhanes.py --cycle 2017-2018 --module DEMO EXAM LAB
   ```
   
   ⚠️ Si la descarga automática falla (común por protecciones del sitio), el script te dará instrucciones claras para descarga manual.
   
   **Opción B - Descarga Manual:**
   - Sitio oficial: https://wwwn.cdc.gov/nchs/nhanes/Default.aspx
   - Selecciona el ciclo (ej: 2015-2016)
   - Descarga los módulos: DEMO, EXAM, LAB, QUEST
   - Coloca los archivos .XPT en `./data/`

2. **Convertir .XPT a CSV:**

   **Opción A - Script Simple (Recomendado):**
   ```bash
   python convertir_nhanes.py
   ```
   
   **Opción B - Script Completo:**
   ```python
   from nhanes_data_converter import convert_xpt_to_csv
   from pathlib import Path
   
   # Convertir todos los .XPT en ./data/
   for xpt_file in Path('./data').glob('*.XPT'):
       convert_xpt_to_csv(xpt_file)
   ```
   
   **Opción C - Manualmente:**
   ```python
   import pandas as pd
   df = pd.read_sas('DEMO_2015_2016.XPT', encoding='utf-8')
   df.to_csv('DEMO_2015_2016.csv', index=False)
   ```

3. **Colocar archivos CSV en `./data/` con el formato:**
   - `DEMO_2007_2008.csv` (o `DEMO_J.csv` para ciclo 2017-2018)
   - `EXAM_2007_2008.csv`
   - `LAB_2007_2008.csv`
   - `QUEST_2007_2008.csv`

**📖 Ver guías completas:**
- `README.md` - Documentación completa
- `QUICK_START.md` - Guía de inicio rápido
- `CONVERSION_DATOS_NHANES.md` - Guía detallada de conversión


---
## 📊 FASE 1: MANEJO DE DATOS NHANES (H0.5 a H4) 
### ⚠️ LA PARTE MÁS CRÍTICA DEL HACKATHON

### 1.1 Entender la estructura NHANES

#### 📖 CONCEPTOS CLAVE DE NHANES:

**Estructura de datos:**
- **SEQN**: ID único de participante (clave para hacer merge)
- **Ciclos**: Períodos de 2 años (2007-2008, 2009-2010, etc.)
- **Módulos separados**: Demographics, Examination, Laboratory, Questionnaire, Dietary

**Pesos muestrales (WEIGHTS):**
- `WTMEC2YR`: Peso para exámenes médicos (2 años)
- `WTINT2YR`: Peso para entrevistas
- `WTDRD1`: Peso para datos dietéticos
- **Regla:** Usar el peso más restrictivo del análisis

**Diseño complejo:**
- `SDMVPSU`: Unidad primaria de muestreo (cluster)
- `SDMVSTRA`: Estrato de muestreo
- Necesario para estadísticas poblacionales correctas

**Variables típicas:**
- Demographics: `RIDAGEYR` (edad), `RIAGENDR` (sexo), `RIDRETH3` (etnia)
- Examination: `BMXWT` (peso), `BMXHT` (altura), `BMXWAIST` (cintura)
- BP: `BPXSY1`, `BPXDI1` (presión arterial)
- Lab: `LBXGH` (A1c), `LBXGLU` (glucosa)
- Questionnaire: `SLQ050` (sueño), `SMQ020` (fumador), `PAQ605` (actividad física)

### 1.2 Función de carga robusta

In [ ]:
CYCLE_TO_LETTER = {
    "2007-2008": "E",
    "2009-2010": "F",
    "2011-2012": "G",
    "2013-2014": "H",
    "2015-2016": "I",
    "2017-2018": "J",
}

def load_nhanes_data(data_dir='./data', cycles=None):
    """
    Carga y merge de datos NHANES por ciclo.
    
    Args:
        data_dir: Directorio con archivos CSV
        cycles: Lista de ciclos a cargar, ej: ['2015-2016']
    
    Returns:
        df: DataFrame consolidado
        metadata: Diccionario con información del merge
    """
    data_path = Path(data_dir)
    
    # Verificar que el directorio existe
    if not data_path.exists():
        print(f"⚠️ ERROR: El directorio {data_path} no existe")
        print(f"   Crea el directorio y coloca los archivos CSV allí")
        print(f"   Estructura esperada:")
        print(f"   {data_path}/")
        print(f"     ├── DEMO_2007_2008.csv")
        print(f"     ├── EXAM_2007_2008.csv")
        print(f"     ├── LAB_2007_2008.csv")
        print(f"     └── QUEST_2007_2008.csv")
        return pd.DataFrame(), {'cycles': [], 'modules': [], 'n_participants': {}}
    
    if cycles is None:
        # Ciclos de entrenamiento
        cycles = ['2015-2016']
    
    all_data = []
    metadata = {'cycles': cycles, 'modules': [], 'n_participants': {}}
    
    for cycle in cycles:
        print(f"\n📁 Cargando ciclo {cycle}...")
        cycle_data = None
        
        # 1. DEMOGRAPHICS (siempre la base)
        # Intentar ambos formatos: DEMO_2017_2018.csv (con guión bajo) y DEMO_J.csv (con letra)
        letter = CYCLE_TO_LETTER.get(cycle, '')
        demo_file = data_path / f"DEMO_{cycle.replace('-', '_')}.csv"
        if not demo_file.exists() and letter:
            # Intentar formato con letra (ej: DEMO_J.csv)
            demo_file = data_path / f"DEMO_{letter}.csv"
        if demo_file.exists():
            try:
                demo = pd.read_csv(demo_file)
                if 'SEQN' not in demo.columns:
                    print(f"  ⚠️ ERROR: Columna SEQN no encontrada en {demo_file}")
                    print(f"     Este archivo no es válido para merge")
                    continue
                demo['CYCLE'] = cycle
                cycle_data = demo
                print(f"  ✓ Demographics: {len(demo):,} registros, {len(demo.columns)} columnas")
            except Exception as e:
                print(f"  ⚠️ ERROR leyendo {demo_file}: {e}")
                continue
        else:
            print(f"  ⚠️ Demographics no encontrado: {demo_file}")
            continue
        
        # 2. EXAMINATION (antropometría y PA)
        # Intentar ambos formatos para EXAM
        exam_file = data_path / f"EXAM_{cycle.replace('-', '_')}.csv"
        if not exam_file.exists() and letter:
            exam_file = data_path / f"EXAM_{letter}.csv"
        if exam_file.exists():
            try:
                exam = pd.read_csv(exam_file)
                if 'SEQN' in exam.columns:
                    cycle_data = cycle_data.merge(exam, on='SEQN', how='left', suffixes=('', '_exam'))
                    print(f"  ✓ Examination: {len(exam):,} registros merged, {len(exam.columns)} columnas")
                else:
                    print(f"  ⚠️ Columna SEQN no encontrada en {exam_file}, saltando merge")
            except Exception as e:
                print(f"  ⚠️ ERROR leyendo {exam_file}: {e}")
        
        # 3. LABORATORY (A1c, glucosa - SOLO PARA LABEL)
        # Intentar ambos formatos para LAB
        lab_file = data_path / f"LAB_{cycle.replace('-', '_')}.csv"
        if not lab_file.exists() and letter:
            lab_file = data_path / f"LAB_{letter}.csv"
        if lab_file.exists():
            try:
                lab = pd.read_csv(lab_file)
                if 'SEQN' in lab.columns:
                    # CRÍTICO: Marcar columnas de lab para no usarlas como features
                    lab_cols = [c for c in lab.columns if c != 'SEQN']
                    renamed_cols = {}
                    for col in lab_cols:
                        if col.startswith('LAB_'):
                            renamed_cols[col] = col
                        else:
                            renamed_cols[col] = f'LAB_{col}'
                    lab = lab.rename(columns=renamed_cols)
                    cycle_data = cycle_data.merge(lab, on='SEQN', how='left')
                    print(f"  ✓ Laboratory: {len(lab):,} registros merged (SOLO PARA LABEL), {len(lab_cols)} columnas de lab")
                else:
                    print(f"  ⚠️ Columna SEQN no encontrada en {lab_file}, saltando merge")
            except Exception as e:
                print(f"  ⚠️ ERROR leyendo {lab_file}: {e}")
        else:
            print(f"  ⚠️ Laboratory no encontrado: {lab_file} (opcional pero recomendado)")
        
        # 4. QUESTIONNAIRE (sueño, actividad, tabaco)
        # Intentar ambos formatos para QUEST
        quest_file = data_path / f"QUEST_{cycle.replace('-', '_')}.csv"
        if not quest_file.exists() and letter:
            quest_file = data_path / f"QUEST_{letter}.csv"
        if quest_file.exists():
            try:
                quest = pd.read_csv(quest_file)
                if 'SEQN' in quest.columns:
                    cycle_data = cycle_data.merge(quest, on='SEQN', how='left', suffixes=('', '_quest'))
                    print(f"  ✓ Questionnaire: {len(quest):,} registros merged, {len(quest.columns)} columnas")
                else:
                    print(f"  ⚠️ Columna SEQN no encontrada en {quest_file}, saltando merge")
            except Exception as e:
                print(f"  ⚠️ ERROR leyendo {quest_file}: {e}")
        else:
            print(f"  ⚠️ Questionnaire no encontrado: {quest_file} (opcional)")
        
        # 5. DIETARY (opcional, nutrición)
        # Intentar ambos formatos para DIET
        diet_file = data_path / f"DIET_{cycle.replace('-', '_')}.csv"
        if not diet_file.exists() and letter:
            diet_file = data_path / f"DIET_{letter}.csv"
        if diet_file.exists():
            try:
                diet = pd.read_csv(diet_file)
                if 'SEQN' in diet.columns:
                    cycle_data = cycle_data.merge(diet, on='SEQN', how='left', suffixes=('', '_diet'))
                    print(f"  ✓ Dietary: {len(diet):,} registros merged, {len(diet.columns)} columnas")
                else:
                    print(f"  ⚠️ Columna SEQN no encontrada en {diet_file}, saltando merge")
            except Exception as e:
                print(f"  ⚠️ ERROR leyendo {diet_file}: {e}")
        
        if cycle_data is not None:
            all_data.append(cycle_data)
            metadata['n_participants'][cycle] = len(cycle_data)
    
    # Concatenar todos los ciclos
    if all_data:
        df = pd.concat(all_data, ignore_index=True)
        print(f"\n✅ TOTAL: {len(df):,} participantes cargados")
        print(f"   Columnas totales: {df.shape[1]}")
        print(f"   Ciclos cargados: {len(all_data)}/{len(cycles)}")
    else:
        print(f"\n⚠️ ADVERTENCIA: No se cargaron datos de ningún ciclo")
        print(f"   Verifica que los archivos CSV estén en {data_path}")
        df = pd.DataFrame()
    
    return df, metadata

TRAINING_CYCLES = ['2015-2016']
TEST_CYCLES = ['2017-2018']

# CARGAR DATOS DE ENTRENAMIENTO
df_train, meta = load_nhanes_data(cycles=TRAINING_CYCLES)

# CARGAR DATOS DE TEST (ciclo ciego)
df_test, meta_test = load_nhanes_data(cycles=TEST_CYCLES)

# Fallback para evitar bloqueos cuando aún no se descarga el ciclo de entrenamiento principal
if df_train.empty:
    print("\n⚠️ No se encontró el ciclo de entrenamiento 2015-2016 en ./data.")
    print("   Usando un split 80/20 del ciclo 2017-2018 solo para exploración.")
    if df_test.empty:
        raise RuntimeError("No hay datos disponibles en ./data. Ejecuta los scripts de descarga antes de continuar.")
    from sklearn.model_selection import train_test_split

    df_train, df_test = train_test_split(
        df_test,
        test_size=0.2,
        random_state=42,
        shuffle=True,
    )
    df_train = df_train.reset_index(drop=True)
    df_test = df_test.reset_index(drop=True)
    meta = {
        'cycles': ['2017-2018 (split_train)'],
        'n_participants': {'2017-2018_split_train': len(df_train)},
        'fallback': True,
    }
    meta_test = {
        'cycles': ['2017-2018 (split_test)'],
        'n_participants': {'2017-2018_split_test': len(df_test)},
        'fallback': True,
    }
else:
    if df_test.empty:
        raise RuntimeError("El ciclo de test 2017-2018 no está disponible. Descárgalo antes de continuar.")

: 

: 

### 1.3 Exploración inicial crítica

In [ ]:
# Verificar distribución por ciclo
print("📊 Distribución por ciclo:")
print(df_train['CYCLE'].value_counts().sort_index())
print(f"\n📊 Test set: {df_test['CYCLE'].value_counts()}")

# Variables clave disponibles
print("\n🔑 Variables clave disponibles:")
key_vars = {
    'Demographics': ['RIDAGEYR', 'RIAGENDR', 'RIDRETH3'],
    'Anthropometry': ['BMXWT', 'BMXHT', 'BMXWAIST', 'BMXBMI'],
    'Blood Pressure': ['BPXSY1', 'BPXSY2', 'BPXDI1', 'BPXDI2'],
    'Laboratory': ['LAB_LBXGH', 'LAB_LBXGLU'],  # Prefijo LAB_
    'Sleep': ['SLD010H', 'SLD012'],
    'Smoking': ['SMQ020', 'SMD030'],
    'Physical Activity': ['PAQ605', 'PAQ620', 'PAD680']
}

for module, vars in key_vars.items():
    available = [v for v in vars if v in df_train.columns]
    print(f"  {module}: {len(available)}/{len(vars)} disponibles")
    if len(available) < len(vars):
        missing = [v for v in vars if v not in df_train.columns]
        print(f"    ⚠️ Faltantes: {missing}")

: 

: 

### 1.4 Limpieza y tipado de datos

In [ ]:
def clean_nhanes_data(df):
    """
    Limpieza estándar de datos NHANES.
    
    Maneja:
    - Valores missing codificados (77777, 99999, etc.)
    - Conversión de tipos
    - Rangos válidos
    """
    df = df.copy()
    
    # 1. Valores missing codificados en NHANES
    missing_codes = [7, 9, 77, 99, 777, 999, 7777, 9999, 77777, 99999, '.', '']
    for col in df.select_dtypes(include=[np.number]).columns:
        df[col] = df[col].replace(missing_codes, np.nan)
    
    # 2. Edad: solo adultos
    if 'RIDAGEYR' in df.columns:
        df = df[df['RIDAGEYR'] >= 18].copy()
        df = df[df['RIDAGEYR'] <= 85].copy()  # Límite superior
    
    # 3. Sexo: 1=M, 2=F
    if 'RIAGENDR' in df.columns:
        df['sex'] = df['RIAGENDR'].map({1: 'M', 2: 'F'})
    
    # 4. Antropometría: rangos razonables
    if 'BMXWT' in df.columns:  # Peso en kg
        df.loc[(df['BMXWT'] < 30) | (df['BMXWT'] > 250), 'BMXWT'] = np.nan
    
    if 'BMXHT' in df.columns:  # Altura en cm
        df.loc[(df['BMXHT'] < 120) | (df['BMXHT'] > 220), 'BMXHT'] = np.nan
    
    if 'BMXWAIST' in df.columns:  # Cintura en cm
        df.loc[(df['BMXWAIST'] < 40) | (df['BMXWAIST'] > 200), 'BMXWAIST'] = np.nan
    
    # 5. Presión arterial: rangos válidos
    for bp_col in ['BPXSY1', 'BPXSY2', 'BPXSY3']:
        if bp_col in df.columns:
            df.loc[(df[bp_col] < 70) | (df[bp_col] > 250), bp_col] = np.nan
    
    for bp_col in ['BPXDI1', 'BPXDI2', 'BPXDI3']:
        if bp_col in df.columns:
            df.loc[(df[bp_col] < 30) | (df[bp_col] > 150), bp_col] = np.nan
    
    print(f"✅ Limpieza completada: {len(df):,} registros válidos")
    return df

# Aplicar limpieza
df_train = clean_nhanes_data(df_train)
df_test = clean_nhanes_data(df_test)

: 

: 

### 1.5 ⚠️ ANTI-FUGA: Identificar columnas de laboratorio

In [ ]:
# CRÍTICO: Identificar todas las columnas de laboratorio
LAB_COLUMNS = [col for col in df_train.columns if col.startswith('LAB_')]

print(f"🚨 COLUMNAS DE LABORATORIO IDENTIFICADAS (NO USAR COMO FEATURES):")
print(f"   Total: {len(LAB_COLUMNS)} columnas")
for col in LAB_COLUMNS[:20]:  # Mostrar primeras 20
    print(f"   - {col}")

# Guardar lista para referencia
with open('LAB_COLUMNS_FORBIDDEN.txt', 'w') as f:
    f.write("\n".join(LAB_COLUMNS))

print("\n✅ Lista guardada en LAB_COLUMNS_FORBIDDEN.txt")

: 

: 

---
## 🎯 FASE 2: CREACIÓN DE LABELS (H4 a H5)

### 2.1 Label A: Alto riesgo de diabetes

In [ ]:
# 🔒 Control global de columnas prohibidas derivadas del label
FORBIDDEN_LABEL_FEATURES: set[str] = set()
FORBIDDEN_LABEL_TOKENS: set[str] = {
    "lbxgh",
    "lbxglu",
    "hba1c",
    "a1c",
    "glucose",
    "glu",
}

def register_forbidden_label_features(columns: list[str]) -> None:
    """Registra columnas y tokens que no pueden usarse como features."""
    global FORBIDDEN_LABEL_FEATURES, FORBIDDEN_LABEL_TOKENS
    for col in columns:
        if not col:
            continue
        col_str = str(col)
        FORBIDDEN_LABEL_FEATURES.add(col_str)
        col_lower = col_str.lower()
        FORBIDDEN_LABEL_TOKENS.update({
            col_lower,
            col_lower.replace("lab_", ""),
            col_lower.replace("_", ""),
            col_lower.split("lab_")[-1],
        })

def feature_has_forbidden_token(feature_name: str) -> bool:
    """Detecta si un nombre de feature contiene tokens prohibidos del label."""
    name = str(feature_name).lower()
    return any(token and token in name for token in FORBIDDEN_LABEL_TOKENS)



: 

: 

In [ ]:
# ✅ Features permitidas explícitamente (demografía + estilo de vida)
ALLOWED_FEATURES: list[str] = [
    # Demográficas
    'age', 'age_squared', 'sex_male',
    # Antropometría
    'bmi', 'bmi_squared', 'waist_height_ratio', 'waist_height_ratio_squared',
    'high_waist_height_ratio', 'central_obesity', 'high_risk_profile',
    # Estilo de vida
    'sleep_hours', 'poor_sleep',
    'cigarettes_per_day', 'current_smoker', 'ever_smoker',
    'total_active_days', 'meets_activity_guidelines', 'sedentary_flag',
    'lifestyle_risk_score',
    # Interacciones y derivados válidos
    'bmi_age_interaction', 'waist_age_interaction',
    'bmi_age_sex_interaction', 'obesity_sedentary_combo',
    'age_poor_sleep', 'triple_risk',
]



: 

: 

In [ ]:
def create_diabetes_label(df):
    """
    Label A: Alto riesgo de diabetes usando HbA1c y/o glucosa en ayunas.

    Basado en ADA Standards of Care 2024 y restricciones del desafío:
      - HbA1c >= 6.0% → Riesgo alto (incluye prediabetes avanzada y diabetes)
      - Glucosa en ayunas >= 110 mg/dL → Riesgo alto (IFG moderado/alto + diabetes)

    Returns:
        tuple[pd.DataFrame, list[str]]: dataframe filtrado y columnas prohibidas desde el label
    """
    df = df.copy()

    def _resolve_column(df, candidates):
        for name in candidates:
            if name in df.columns:
                return name
        return None

    a1c_col = _resolve_column(df, ['LAB_LBXGH', 'LAB_LAB_LBXGH', 'LBXGH', 'HBA1C', 'A1C'])
    glucose_col = _resolve_column(df, ['LAB_LBXGLU', 'LAB_LAB_LBXGLU', 'LBXGLU', 'GLUCOSE', 'GLU_FAST'])

    df['label_diabetes'] = 0

    has_a1c = a1c_col is not None
    has_glucose = glucose_col is not None

    if not has_a1c and not has_glucose:
        print("⚠️ ADVERTENCIA: No se encontraron columnas LAB_LBXGH o LAB_LBXGLU")
        print("   Verifica que los archivos LAB_*.csv se hayan combinado correctamente")
        return df, []

    forbidden_from_label = []

    if has_a1c:
        valid_a1c = df[a1c_col].notna()
        n_a1c = (valid_a1c & (df[a1c_col] >= 6.0)).sum()
        df.loc[valid_a1c & (df[a1c_col] >= 6.0), 'label_diabetes'] = 1
        forbidden_from_label.extend([a1c_col, 'LBXGH', 'HBA1C', 'A1C'])
        print(f"  HbA1c >= 6.0%: {n_a1c:,} casos (columna: {a1c_col})")
    else:
        print("  ⚠️ Ninguna columna de HbA1c encontrada en el dataset")

    if has_glucose:
        valid_glu = df[glucose_col].notna()
        n_glu = (valid_glu & (df[glucose_col] >= 110)).sum()
        df.loc[valid_glu & (df[glucose_col] >= 110), 'label_diabetes'] = 1
        forbidden_from_label.extend([glucose_col, 'LBXGLU', 'GLUCOSE', 'GLU_FAST'])
        print(f"  Glucosa >= 110 mg/dL: {n_glu:,} casos (columna: {glucose_col})")
    else:
        print("  ⚠️ Ninguna columna de glucosa en ayunas encontrada en el dataset")

    lab_cols = []
    if has_a1c:
        lab_cols.append(a1c_col)
    if has_glucose:
        lab_cols.append(glucose_col)

    if lab_cols:
        has_lab_data = df[lab_cols].notna().any(axis=1)
        df = df[has_lab_data].copy()
    else:
        print("⚠️ ADVERTENCIA: No hay datos de laboratorio disponibles para crear la etiqueta")
        return df, forbidden_from_label

    prevalence = df['label_diabetes'].mean()
    print("\n✅ Label Diabetes creado")
    print(f"   Prevalencia: {prevalence:.1%}")
    print(f"   Casos positivos: {df['label_diabetes'].sum():,} / {len(df):,}")

    register_forbidden_label_features(forbidden_from_label)

    return df, forbidden_from_label

# Crear labels
df_train, forbidden_label_features_train = create_diabetes_label(df_train)
df_test, forbidden_label_features_test = create_diabetes_label(df_test)

FORBIDDEN_LABEL_FEATURES.update(forbidden_label_features_train)
FORBIDDEN_LABEL_FEATURES.update(forbidden_label_features_test)


: 

: 

### 2.2 Label B (alternativa): Hipertensión

In [ ]:
# Indicador binario de diabetes utilizable en la UI
for split_name, df_split in [('Train', df_train), ('Test', df_test)]:
    df_split.loc[:, 'has_diabetes'] = df_split['label_diabetes'].map({1: 'Sí', 0: 'No'})
    counts = df_split['has_diabetes'].value_counts().rename('personas')
    percents = (df_split['has_diabetes'].value_counts(normalize=True) * 100).round(1).rename('%')
    summary = pd.concat([counts, percents], axis=1)
    print(f"👤 {split_name}: distribución de has_diabetes")
    print(summary)
    print()

print("📌 Ejemplo de salida (test):")
df_test[['SEQN', 'has_diabetes']].head()



: 

: 

In [ ]:
def create_hypertension_label(df):
    """
    Etiqueta de hipertensión usando mediciones de PA.
    
    Criterios (Estadio 1+):
    - Sistólica >= 130 mmHg, o
    - Diastólica >= 80 mmHg
    
    Usa promedio de 2-3 mediciones disponibles.
    """
    df = df.copy()
    
    # Calcular promedio de mediciones de PA
    sys_cols = [c for c in df.columns if c.startswith('BPXSY') and len(c) > 5 and c[5:].isdigit()]
    dia_cols = [c for c in df.columns if c.startswith('BPXDI') and len(c) > 5 and c[5:].isdigit()]
    
    if not sys_cols and not dia_cols:
        print("⚠️ ADVERTENCIA: No se encontraron columnas de presión arterial (BPXSY* o BPXDI*)")
        print("   Asegúrate de que los archivos EXAM_*.csv estén cargados correctamente")
        return df
    
    # Inicializar columnas
    if sys_cols:
        df['sbp_mean'] = df[sys_cols].mean(axis=1)
        print(f"  Columnas sistólica encontradas: {sys_cols}")
    else:
        df['sbp_mean'] = np.nan
        print("  ⚠️ No se encontraron columnas de presión sistólica")
    
    if dia_cols:
        df['dbp_mean'] = df[dia_cols].mean(axis=1)
        print(f"  Columnas diastólica encontradas: {dia_cols}")
    else:
        df['dbp_mean'] = np.nan
        print("  ⚠️ No se encontraron columnas de presión diastólica")
    
    # Criterio de hipertensión (solo si tenemos al menos una medida)
    df['label_hypertension'] = 0
    has_bp_data = df[['sbp_mean', 'dbp_mean']].notna().any(axis=1)
    
    if has_bp_data.any():
        # Aplicar criterio solo donde hay datos
        mask = (df['sbp_mean'].notna() & (df['sbp_mean'] >= 130)) | \
               (df['dbp_mean'].notna() & (df['dbp_mean'] >= 80))
        df.loc[mask, 'label_hypertension'] = 1
    
    # Remover casos sin datos de PA
    df = df[has_bp_data].copy()
    
    if len(df) > 0:
        prevalence = df['label_hypertension'].mean()
        print(f"\n✅ Label Hipertensión creado:")
        print(f"   Prevalencia: {prevalence:.1%}")
        print(f"   n = {df['label_hypertension'].sum():,} / {len(df):,}")
    else:
        print("\n⚠️ ADVERTENCIA: No hay datos de presión arterial disponibles")
    
    return df

# Opcional: crear también label de hipertensión
df_train = create_hypertension_label(df_train)
df_test = create_hypertension_label(df_test)

: 

: 

---
## 🔧 FASE 3: INGENIERÍA DE FEATURES (H5 a H7)

### 3.1 Features demográficas y antropométricas

In [ ]:
# 🔄 Redefinir engineer_features para usar solo ALLOWED_FEATURES

def engineer_features(df):
    """Crea features derivadas válidas basadas en demografía y estilo de vida."""
    df = df.copy()

    # 1. ANTROPOMETRÍA
    if {'BMXWT', 'BMXHT'} <= set(df.columns):
        df['bmi'] = df['BMXWT'] / ((df['BMXHT'] / 100) ** 2)
        df['bmi_category'] = pd.cut(
            df['bmi'],
            bins=[0, 18.5, 25, 30, 100],
            labels=['underweight', 'normal', 'overweight', 'obese']
        )
    if {'BMXWAIST', 'BMXHT'} <= set(df.columns):
        df['waist_height_ratio'] = df['BMXWAIST'] / df['BMXHT']
        df['high_waist_height_ratio'] = (df['waist_height_ratio'] >= 0.5).astype(int)

    # 2. EDAD
    if 'RIDAGEYR' in df.columns:
        df['age'] = df['RIDAGEYR']
        df['age_group'] = pd.cut(df['age'], bins=[0, 30, 45, 60, 100], labels=['18-30', '31-45', '46-60', '60+'])
        df['age_squared'] = df['age'] ** 2

    # 3. SEXO
    if 'sex' in df.columns:
        df['sex_male'] = (df['sex'] == 'M').astype(int)

    # 4. SUEÑO
    for col in ['SLD010H', 'SLD012']:
        if col in df.columns:
            df['sleep_hours'] = df[col]
            df['poor_sleep'] = ((df['sleep_hours'] < 7) | (df['sleep_hours'] > 9)).astype(int)
            break

    # 5. TABAQUISMO
    if 'SMQ020' in df.columns:
        df['ever_smoker'] = (df['SMQ020'] == 1).astype(int)
    if 'SMD030' in df.columns:
        df['cigarettes_per_day'] = df['SMD030']
        df['current_smoker'] = (df['cigarettes_per_day'] > 0).astype(int)

    # 6. ACTIVIDAD FÍSICA
    if {'PAQ605', 'PAQ620'} <= set(df.columns):
        df['total_active_days'] = df['PAQ605'].fillna(0) + df['PAQ620'].fillna(0)
        df['meets_activity_guidelines'] = (df['total_active_days'] >= 5).astype(int)
        df['sedentary_flag'] = (df['meets_activity_guidelines'] == 0).astype(int)

    # 7. INTERACCIONES
    if {'bmi', 'age'} <= set(df.columns):
        df['bmi_age_interaction'] = df['bmi'] * df['age']
        df['high_risk_profile'] = ((df['bmi'] >= 30) & (df['age'] >= 45)).astype(int)
    if {'waist_height_ratio', 'age'} <= set(df.columns):
        df['waist_age_interaction'] = df['waist_height_ratio'] * df['age']

    # 8. OBESIDAD CENTRAL
    if {'BMXWAIST', 'sex'} <= set(df.columns):
        df['central_obesity'] = np.where(
            df['BMXWAIST'].notna(),
            np.where(
                ((df['sex'] == 'M') & (df['BMXWAIST'] >= 102)) |
                ((df['sex'] == 'F') & (df['BMXWAIST'] >= 88)),
                1,
                0
            ),
            np.nan
        )

    # 9. PUNTAJE DE ESTILO DE VIDA
    lifestyle_components = []
    for col in ['current_smoker', 'poor_sleep', 'sedentary_flag']:
        if col in df.columns:
            lifestyle_components.append(df[col].fillna(0))
    if lifestyle_components:
        df['lifestyle_risk_score'] = np.vstack(lifestyle_components).sum(axis=0)

    # 10. FEATURES AVANZADAS
    if {'bmi', 'age', 'sex_male'} <= set(df.columns):
        df['bmi_age_sex_interaction'] = df['bmi'] * df['age'] * df['sex_male']
    if 'waist_height_ratio' in df.columns:
        df['waist_height_ratio_squared'] = df['waist_height_ratio'] ** 2
    if 'bmi' in df.columns:
        df['bmi_squared'] = df['bmi'] ** 2
    if {'central_obesity', 'sedentary_flag'} <= set(df.columns):
        df['obesity_sedentary_combo'] = df['central_obesity'] * df['sedentary_flag']
    if {'age', 'poor_sleep'} <= set(df.columns):
        df['age_poor_sleep'] = df['age'] * df['poor_sleep']
    if {'central_obesity', 'age', 'current_smoker'} <= set(df.columns):
        df['triple_risk'] = df['central_obesity'] * (df['age'] >= 45).astype(int) * df['current_smoker']

    # Garantizar que todas las features permitidas existan
    for feature in ALLOWED_FEATURES:
        if feature not in df.columns:
            df[feature] = np.nan

    print(f"✅ Features creadas: {df.shape[1]} columnas totales")

    available_features = [f for f in ALLOWED_FEATURES if f in df.columns]
    print(f"\n📊 Features numéricas disponibles: {len(available_features)}")
    for feat in available_features:
        missing_pct = df[feat].isna().mean() * 100
        print(f"   - {feat}: {missing_pct:.1f}% missing")

    return df



: 

: 

In [ ]:
def engineer_features(df):
    """
    Crea features derivadas SOLO de variables permitidas.
    
    NO usar columnas de laboratorio (LAB_*).
    """
    df = df.copy()
    
    # 1. ANTROPOMETRÍA
    # IMC (Body Mass Index)
    if 'BMXWT' in df.columns and 'BMXHT' in df.columns:
        df['bmi'] = df['BMXWT'] / ((df['BMXHT'] / 100) ** 2)
        # Categorías de IMC
        df['bmi_category'] = pd.cut(df['bmi'], 
                                     bins=[0, 18.5, 25, 30, 100],
                                     labels=['underweight', 'normal', 'overweight', 'obese'])
    
    # Ratio cintura-altura (mejor predictor que IMC)
    if 'BMXWAIST' in df.columns and 'BMXHT' in df.columns:
        df['waist_height_ratio'] = df['BMXWAIST'] / df['BMXHT']
        # Riesgo si >= 0.5
        df['high_waist_height_ratio'] = (df['waist_height_ratio'] >= 0.5).astype(int)
    
    # 2. EDAD
    if 'RIDAGEYR' in df.columns:
        df['age'] = df['RIDAGEYR']
        # Grupos etarios
        df['age_group'] = pd.cut(df['age'], 
                                  bins=[0, 30, 45, 60, 100],
                                  labels=['18-30', '31-45', '46-60', '60+'])
        # Edad al cuadrado (relación no lineal)
        df['age_squared'] = df['age'] ** 2
    
    # 3. SEXO (ya mapeado en limpieza)
    if 'sex' in df.columns:
        df['sex_male'] = (df['sex'] == 'M').astype(int)
    
    # 4. SUEÑO
    sleep_cols = ['SLD010H', 'SLD012']  # Horas de sueño
    for col in sleep_cols:
        if col in df.columns:
            df['sleep_hours'] = df[col]
            # Sueño insuficiente (<7h) o excesivo (>9h)
            df['poor_sleep'] = ((df['sleep_hours'] < 7) | (df['sleep_hours'] > 9)).astype(int)
            break
    
    # 5. TABAQUISMO
    if 'SMQ020' in df.columns:  # ¿Ha fumado 100+ cigarrillos?
        df['ever_smoker'] = (df['SMQ020'] == 1).astype(int)

    if 'SMD030' in df.columns:  # Cigarrillos por día
        df['cigarettes_per_day'] = df['SMD030']
        df['current_smoker'] = (df['cigarettes_per_day'] > 0).astype(int)

    # 6. ACTIVIDAD FÍSICA
    if 'PAQ605' in df.columns and 'PAQ620' in df.columns:
        df['total_active_days'] = df['PAQ605'].fillna(0) + df['PAQ620'].fillna(0)
        # Cumple recomendaciones (150+ min/semana ≈ 5 días)
        df['meets_activity_guidelines'] = (df['total_active_days'] >= 5).astype(int)
        df['sedentary_flag'] = (df['meets_activity_guidelines'] == 0).astype(int)

    # 7. INTERACCIONES IMPORTANTES
    if 'bmi' in df.columns and 'age' in df.columns:
        df['bmi_age_interaction'] = df['bmi'] * df['age']
        df['high_risk_profile'] = ((df['bmi'] >= 30) & (df['age'] >= 45)).astype(int)

    if 'waist_height_ratio' in df.columns and 'age' in df.columns:
        df['waist_age_interaction'] = df['waist_height_ratio'] * df['age']

    # 8. OBESIDAD CENTRAL SEGÚN SEXO
    if 'BMXWAIST' in df.columns and 'sex' in df.columns:
        df['central_obesity'] = np.where(
            df['BMXWAIST'].notna(),
            np.where(
                ((df['sex'] == 'M') & (df['BMXWAIST'] >= 102)) |
                ((df['sex'] == 'F') & (df['BMXWAIST'] >= 88)),
                1,
                0
            ),
            np.nan
        )

    # 9. PUNTAJE DE ESTILO DE VIDA
    lifestyle_components = []
    if 'current_smoker' in df.columns:
        lifestyle_components.append(df['current_smoker'].fillna(0))
    if 'poor_sleep' in df.columns:
        lifestyle_components.append(df['poor_sleep'].fillna(0))
    if 'sedentary_flag' in df.columns:
        lifestyle_components.append(df['sedentary_flag'].fillna(0))

    if lifestyle_components:
        df['lifestyle_risk_score'] = np.vstack(lifestyle_components).sum(axis=0)

    # 10. FEATURES ADICIONALES AVANZADAS (Para mejorar AUROC)
    # Interacciones de 3er orden
    if 'bmi' in df.columns and 'age' in df.columns and 'sex_male' in df.columns:
        df['bmi_age_sex_interaction'] = df['bmi'] * df['age'] * df['sex_male']
    
    # Cintura cuadrática (relación no lineal)
    if 'waist_height_ratio' in df.columns:
        df['waist_height_ratio_squared'] = df['waist_height_ratio'] ** 2
    
    # BMI cuadrático
    if 'bmi' in df.columns:
        df['bmi_squared'] = df['bmi'] ** 2
    
    # Perfil de riesgo compuesto
    if 'central_obesity' in df.columns and 'sedentary_flag' in df.columns:
        df['obesity_sedentary_combo'] = df['central_obesity'] * df['sedentary_flag']
    
    # Edad y sueño
    if 'age' in df.columns and 'poor_sleep' in df.columns:
        df['age_poor_sleep'] = df['age'] * df['poor_sleep']
    
    # Obesidad, edad y tabaco (triple riesgo)
    if 'central_obesity' in df.columns and 'age' in df.columns and 'current_smoker' in df.columns:
        df['triple_risk'] = df['central_obesity'] * (df['age'] >= 45).astype(int) * df['current_smoker']

    print(f"✅ Features creadas: {df.shape[1]} columnas totales")

    # Listar nuevas features numéricas
    new_features = ['bmi', 'waist_height_ratio', 'central_obesity', 'age', 'age_squared',
                    'sleep_hours', 'cigarettes_per_day', 'total_active_days', 'lifestyle_risk_score',
                    'bmi_age_interaction', 'waist_age_interaction', 'high_risk_profile', 'sedentary_flag',
                    'bmi_age_sex_interaction', 'waist_height_ratio_squared', 'bmi_squared',
                    'obesity_sedentary_combo', 'age_poor_sleep', 'triple_risk']
    available_features = [f for f in new_features if f in df.columns]

    print(f"\n📊 Features numéricas disponibles: {len(available_features)}")
    for feat in available_features:
        if feat in df.columns:
            missing_pct = df[feat].isna().mean() * 100
            print(f"   - {feat}: {missing_pct:.1f}% missing")

    return df

# Aplicar ingeniería de features
df_train = engineer_features(df_train)
df_test = engineer_features(df_test)

: 

: 

### 3.2 Selección final de features (sin fuga)

In [ ]:
# 🔒 Redefinir select_features_no_leakage usando ALLOWED_FEATURES

def select_features_no_leakage(df, target='label_diabetes'):
    """Selecciona features válidas y aplica chequeos estrictos anti-fuga."""
    candidate_features = [f for f in ALLOWED_FEATURES if f in df.columns]

    lab_features = [f for f in candidate_features if f.startswith('LAB_')]
    if lab_features:
        raise ValueError(f"🚨 FUGA DE DATOS DETECTADA: {lab_features}")

    forbidden_direct = [f for f in candidate_features if f in FORBIDDEN_LABEL_FEATURES]
    forbidden_tokens = [f for f in candidate_features if feature_has_forbidden_token(f)]
    if forbidden_direct or forbidden_tokens:
        raise ValueError(
            "🚨 FUGA DE DATOS DETECTADA por label: "
            f"Directas={forbidden_direct}, Derivadas={forbidden_tokens}"
        )

    print(f"✅ Features seleccionadas: {len(candidate_features)}")
    print(f"   Target: {target}")

    X = df[candidate_features].copy()
    y = df[target].copy()

    valid_idx = y.notna()
    X = X[valid_idx]
    y = y[valid_idx]

    print(f"\n📊 Dataset final: {len(X):,} registros")
    print(f"   Prevalencia: {y.mean():.1%}")
    print(f"   Clase positiva: {int(y.sum()):,} | Clase negativa: {int(len(y) - y.sum()):,}")

    return X, y, candidate_features



: 

: 

In [ ]:
def legacy_select_features_no_leakage(df, target='label_diabetes'):
    """
    Selecciona features finales SIN columnas de laboratorio ni derivadas del label.
    
    CRÍTICO: Valida que no haya fuga de datos.
    """
    # Features candidatas (ajustar según disponibilidad)
    candidate_features = [
        # Demográficas
        'age', 'age_squared', 'sex_male',

        # Antropometría
        'bmi', 'bmi_squared', 'waist_height_ratio', 'waist_height_ratio_squared',
        'high_waist_height_ratio', 'central_obesity', 'high_risk_profile',

        # Estilo de vida
        'sleep_hours', 'poor_sleep',
        'cigarettes_per_day', 'current_smoker', 'ever_smoker',
        'total_active_days', 'meets_activity_guidelines', 'sedentary_flag',
        'lifestyle_risk_score',

        # Interacciones de 2do orden
        'bmi_age_interaction', 'waist_age_interaction',
        
        # Interacciones avanzadas (3er orden y cuadráticas)
        'bmi_age_sex_interaction', 'obesity_sedentary_combo',
        'age_poor_sleep', 'triple_risk',

        # Presión arterial (si el label NO es hipertensión)
        # 'sbp_mean', 'dbp_mean'  # Descomentar si label = diabetes
    ]
    
    # Filtrar features disponibles
    features = [f for f in candidate_features if f in df.columns]
    
    # VALIDACIÓN ANTI-FUGA
    lab_features = [f for f in features if f.startswith('LAB_')]
    if lab_features:
        raise ValueError(f"🚨 FUGA DE DATOS DETECTADA: {lab_features}")
    
    forbidden_direct = [f for f in features if f in FORBIDDEN_LABEL_FEATURES]
    forbidden_tokens = [f for f in features if feature_has_forbidden_token(f)]
    if forbidden_direct or forbidden_tokens:
        raise ValueError(
            "🚨 FUGA DE DATOS DETECTADA por label: "
            f"Directas={forbidden_direct}, Derivadas={forbidden_tokens}"
        )
    
    print(f"✅ Features seleccionadas: {len(features)}")
    print(f"   Target: {target}")
    
    # Preparar X, y
    X = df[features].copy()
    y = df[target].copy()
    
    # Remover filas con target missing
    valid_idx = y.notna()
    X = X[valid_idx]
    y = y[valid_idx]
    
    print(f"\n📊 Dataset final: {len(X):,} registros")
    print(f"   Prevalencia: {y.mean():.1%}")
    print(f"   Clase positiva: {int(y.sum()):,} | Clase negativa: {int(len(y) - y.sum()):,}")
    
    return X, y, features

# Preparar datos finales
X_train, y_train, feature_names = select_features_no_leakage(df_train, target='label_diabetes')
X_test, y_test, _ = select_features_no_leakage(df_test, target='label_diabetes')

: 

: 

---
## 🤖 FASE 4: MODELO ML (H7 a H10)

### 4.1 Baseline: Regresión Logística

In [ ]:
# Validación anti-fuga (anti-leakage)
LAB_COLUMNS_TRAIN = [col for col in df_train.columns if col.startswith('LAB_')]
LAB_COLUMNS_TEST = [col for col in df_test.columns if col.startswith('LAB_')]
print(f"🚨 Columnas de laboratorio detectadas en df_train: {len(LAB_COLUMNS_TRAIN)}")
print(f"🚨 Columnas de laboratorio detectadas en df_test: {len(LAB_COLUMNS_TEST)}")
if LAB_COLUMNS_TRAIN:
    print(f"   Ejemplo df_train LAB_: {LAB_COLUMNS_TRAIN[:10]}")
if LAB_COLUMNS_TEST:
    print(f"   Ejemplo df_test LAB_: {LAB_COLUMNS_TEST[:10]}")

lab_features_detected_train = [feat for feat in X_train.columns if feat.startswith('LAB_')]
lab_features_detected_test = [feat for feat in X_test.columns if feat.startswith('LAB_')]
lab_features_in_feature_names = [feat for feat in feature_names if feat.startswith('LAB_')]

if lab_features_detected_train or lab_features_detected_test or lab_features_in_feature_names:
    raise ValueError(
        "🚨 FUGA DE DATOS: se detectaron variables de laboratorio en el set de features\n"
        f"  Train: {lab_features_detected_train}\n"
        f"  Test: {lab_features_detected_test}\n"
        f"  Feature names: {lab_features_in_feature_names}"
    )

print("✅ Validación anti-fuga superada: ninguna feature entrenable proviene de laboratorio")


: 

: 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss
from pathlib import Path
import json
import joblib

# Pipeline con imputación y escalado
pipeline_lr = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced'))
])

# Entrenar
print("🔄 Entrenando Logistic Regression...")
pipeline_lr.fit(X_train, y_train)

# Predicciones
y_pred_proba_lr = pipeline_lr.predict_proba(X_test)[:, 1]

# Métricas
auroc_lr = roc_auc_score(y_test, y_pred_proba_lr)
auprc_lr = average_precision_score(y_test, y_pred_proba_lr)
brier_lr = brier_score_loss(y_test, y_pred_proba_lr)

print(f"\n✅ BASELINE - Logistic Regression:")
print(f"   AUROC: {auroc_lr:.4f}")
print(f"   AUPRC: {auprc_lr:.4f}")
print(f"   Brier Score: {brier_lr:.4f}")

# Persistir métricas
metrics_lr = {
    "auroc": float(auroc_lr),
    "auprc": float(auprc_lr),
    "brier": float(brier_lr),
    "n_train": int(len(y_train)),
    "n_test": int(len(y_test)),
}
Path('reports').mkdir(exist_ok=True)
with open('reports/metrics_logreg.json', 'w', encoding='utf-8') as f:
    json.dump(metrics_lr, f, indent=2)
print("💾 Métricas baseline guardadas en reports/metrics_logreg.json")

# Guardar modelo
joblib.dump(pipeline_lr, 'model_baseline_lr.pkl')
print("\n💾 Modelo guardado: model_baseline_lr.pkl")

: 

: 

In [ ]:
# Validación integral del baseline (Logistic Regression)
print("🔎 Validación baseline - Logistic Regression")
required_vars = ['auroc_lr', 'auprc_lr', 'brier_lr', 'y_train', 'y_test']
missing_vars = [var for var in required_vars if var not in globals()]
if missing_vars:
    raise RuntimeError(f"Variables requeridas ausentes después del entrenamiento: {missing_vars}")

train_counts = y_train.value_counts().sort_index()
test_counts = y_test.value_counts().sort_index()
class_balance_ok = len(train_counts) >= 2 and len(test_counts) >= 2

threshold_checks = {
    'AUROC ≥ 0.80': auroc_lr >= 0.80,
    'Brier ≤ 0.12': brier_lr <= 0.12,
}

print("\n📈 Métricas baseline:")
print(f"   AUROC: {auroc_lr:.4f}")
print(f"   AUPRC: {auprc_lr:.4f}")
print(f"   Brier Score: {brier_lr:.4f}")

print("\n👥 Distribución de clases")
print("   Train:")
for label, count in train_counts.items():
    pct = (count / len(y_train)) * 100
    print(f"     Clase {label}: {count:,} ({pct:.1f}%)")
print("   Test:")
for label, count in test_counts.items():
    pct = (count / len(y_test)) * 100
    print(f"     Clase {label}: {count:,} ({pct:.1f}%)")

warnings = []
for description, passed in threshold_checks.items():
    icon = "✅" if passed else "⚠️"
    print(f"{icon} {description}")
    if not passed:
        warnings.append(description)

if class_balance_ok:
    print("✅ Ambos conjuntos contienen clases positiva y negativa")
else:
    warnings.append('Distribución de clases incompleta en train o test')
    print("⚠️ Distribución de clases incompleta en train o test")

baseline_validation = {
    'auroc': auroc_lr,
    'auprc': auprc_lr,
    'brier': brier_lr,
    'class_balance_ok': class_balance_ok,
    'thresholds': threshold_checks,
    'warnings': warnings,
}

if warnings:
    print("\n⚠️ Advertencias detectadas:")
    for note in warnings:
        print(f"   - {note}")
else:
    print("\n✅ Baseline cumple los umbrales definidos por el desafío")


: 

: 

### 4.2 Modelo avanzado: XGBoost

In [ ]:
from xgboost import XGBClassifier

# Imputación previa
imputer = SimpleImputer(strategy='median')
X_train_imp = pd.DataFrame(
    imputer.fit_transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)
X_test_imp = pd.DataFrame(
    imputer.transform(X_test),
    columns=X_test.columns,
    index=X_test.index
)

# Desbalance
pos_count = int((y_train == 1).sum())
neg_count = int((y_train == 0).sum())
if pos_count == 0:
    raise ValueError("⚠️ No hay casos positivos en el conjunto de entrenamiento")
scale_pos_weight = neg_count / pos_count
print(f"   Desbalance: {neg_count:,} negativos / {pos_count:,} positivos")
print(f"   scale_pos_weight: {scale_pos_weight:.2f}")

# Modelo XGBoost competitivo
model_xgb = XGBClassifier(
    n_estimators=600,
    learning_rate=0.03,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='binary:logistic',
    eval_metric=['auc', 'aucpr', 'logloss'],
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_jobs=-1,
)

print("🔄 Entrenando XGBoost competitivo...")
model_xgb.fit(
    X_train_imp,
    y_train,
    eval_set=[(X_train_imp, y_train), (X_test_imp, y_test)],
    # early_stopping_rounds=50,
    verbose=False,
)

# Predicciones
y_pred_proba_xgb = model_xgb.predict_proba(X_test_imp)[:, 1]

# Métricas
auroc_xgb = roc_auc_score(y_test, y_pred_proba_xgb)
auprc_xgb = average_precision_score(y_test, y_pred_proba_xgb)
brier_xgb = brier_score_loss(y_test, y_pred_proba_xgb)

print("\n✅ XGBoost competitivo:")
print(f"   AUROC: {auroc_xgb:.4f}")
print(f"   AUPRC: {auprc_xgb:.4f}")
print(f"   Brier Score: {brier_xgb:.4f}")

feature_importance = pd.DataFrame({
    'feature': feature_names,
    'importance': model_xgb.feature_importances_
}).sort_values('importance', ascending=False)
print("\n📊 Top 10 Features:")
print(feature_importance.head(10).to_string(index=False))

# Persistir métricas y artefactos
metrics_xgb = {
    "auroc": float(auroc_xgb),
    "auprc": float(auprc_xgb),
    "brier": float(brier_xgb),
    "best_iteration": int(getattr(model_xgb, 'best_iteration', model_xgb.n_estimators)),
    "n_train": int(len(y_train)),
    "n_test": int(len(y_test)),
    "scale_pos_weight": float(scale_pos_weight),
}
Path('reports').mkdir(exist_ok=True)
with open('reports/metrics_xgb.json', 'w', encoding='utf-8') as f:
    json.dump(metrics_xgb, f, indent=2)
print("💾 Métricas XGBoost guardadas en reports/metrics_xgb.json")

joblib.dump(imputer, 'imputer.pkl')
joblib.dump({
    "imputer": imputer,
    "model": model_xgb,
    "feature_names": feature_names,
}, 'model_xgb_bundle.pkl')
joblib.dump(model_xgb, 'model_xgboost.pkl')
print("💾 Bundle guardado en model_xgb_bundle.pkl y modelo base en model_xgboost.pkl")

: 

: 

In [ ]:
import shap

print("🧮 Computando valores SHAP para XGBoost...")
explainer = shap.TreeExplainer(model_xgb)
shap_values = explainer.shap_values(X_test_imp)

shap_importance = np.abs(shap_values).mean(axis=0)
shap_df = pd.DataFrame({
    'feature': feature_names,
    'shap_importance': shap_importance,
    'xgb_importance': model_xgb.feature_importances_
}).sort_values('shap_importance', ascending=False)

print("\n📊 Top 10 features por importancia SHAP:")
print(shap_df.head(10).to_string(index=False))

top_features_global = shap_df.head(5)['feature'].tolist()
with open('reports/top_drivers.json', 'w', encoding='utf-8') as f:
    json.dump({'top_features': top_features_global}, f, indent=2, ensure_ascii=False)
print("💾 Top drivers globales guardados en reports/top_drivers.json")

# Resumen global
title = 'SHAP Summary - Impacto global de features'
shap.summary_plot(
    shap_values,
    X_test_imp,
    feature_names=feature_names,
    show=False,
    max_display=15
)
plt.title(title)
plt.tight_layout()
plt.savefig('shap_summary.png', dpi=300, bbox_inches='tight')
plt.show()
print("✅ SHAP summary guardado en shap_summary.png")

# Guardar ranking global para reportes
Path('reports').mkdir(exist_ok=True)
shap_df.to_csv('reports/shap_feature_importance.csv', index=False)
print("💾 Importancia SHAP global guardada en reports/shap_feature_importance.csv")

# Explicaciones locales

def get_top_drivers(idx: int, n: int = 5):
    shap_vals = shap_values[idx]
    top_idx = np.argsort(np.abs(shap_vals))[-n:][::-1]
    drivers = []
    for i in top_idx:
        drivers.append({
            'feature': feature_names[i],
            'feature_value': float(X_test_imp.iloc[idx, i]),
            'shap_value': float(shap_vals[i]),
            'impact': 'aumenta' if shap_vals[i] > 0 else 'reduce'
        })
    return drivers

# Ejemplo de explicación local
example_idx = 0
example_drivers = get_top_drivers(example_idx)
print(f"\n🧬 Explicación local para la fila de test {example_idx}:")
for driver in example_drivers:
    print(f"  {driver['feature']}: valor {driver['feature_value']:.3f} → {driver['impact']} el riesgo (SHAP {driver['shap_value']:+.4f})")

local_explanations_df = pd.DataFrame(example_drivers)
local_explanations_df.to_csv('reports/shap_example_drivers.csv', index=False)
print("💾 Explicación local de ejemplo guardada en reports/shap_example_drivers.csv")


: 

: 

### 4.3 Curvas de calibración

In [ ]:
from sklearn.calibration import calibration_curve, CalibratedClassifierCV

calibration_candidates = {
    'sin_calibrar': {
        'model': model_xgb,
        'pred': y_pred_proba_xgb,
        'brier': brier_xgb,
    }
}

for method in ['isotonic', 'sigmoid']:
    print(f"\n   Probando calibración {method}...")
    calibrator = CalibratedClassifierCV(model_xgb, method=method, cv='prefit')
    calibrator.fit(X_train_imp, y_train)
    pred = calibrator.predict_proba(X_test_imp)[:, 1]
    brier = brier_score_loss(y_test, pred)
    print(f"      Brier Score ({method}): {brier:.4f}")
    calibration_candidates[method] = {
        'model': calibrator,
        'pred': pred,
        'brier': brier,
    }

best_method = min(calibration_candidates, key=lambda m: calibration_candidates[m]['brier'])
model_xgb_calibrated = calibration_candidates[best_method]['model']
y_pred_proba_xgb = calibration_candidates[best_method]['pred']
brier_xgb = calibration_candidates[best_method]['brier']

print(f"\n✅ Método seleccionado: {best_method} (Brier={brier_xgb:.4f})")
if best_method == 'sin_calibrar':
    print("   No se obtuvo mejora sobre el modelo original")
else:
    print("   Calibración aplicada y lista para producción")

prob_true_lr, prob_pred_lr = calibration_curve(y_test, y_pred_proba_lr, n_bins=10)
prob_true_xgb, prob_pred_xgb = calibration_curve(y_test, y_pred_proba_xgb, n_bins=10)

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
ax.plot([0, 1], [0, 1], 'k--', label='Perfect calibration')
ax.plot(prob_pred_lr, prob_true_lr, 'o-', label=f'Logistic Regression (Brier={brier_lr:.4f})')
ax.plot(prob_pred_xgb, prob_true_xgb, 's-', label=f'XGBoost ({best_method}) (Brier={brier_xgb:.4f})')
ax.set_xlabel('Mean predicted probability', fontsize=12)
ax.set_ylabel('Fraction of positives', fontsize=12)
ax.set_title('Calibration Curves', fontsize=14, fontweight='bold')
ax.legend(loc='upper left')
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('calibration_curves.png', dpi=300, bbox_inches='tight')
plt.show()

# Guardar modelo calibrado
joblib.dump({
    "imputer": imputer,
    "model": model_xgb_calibrated,
    "feature_names": feature_names,
    "calibration": best_method,
}, 'model_xgb_calibrated.pkl')
print("💾 Modelo calibrado guardado en model_xgb_calibrated.pkl")

# Actualizar métricas con información de calibración
metrics_path = Path('reports/metrics_xgb.json')
if metrics_path.exists():
    with open(metrics_path, 'r', encoding='utf-8') as f:
        metrics_xgb_existing = json.load(f)
else:
    metrics_xgb_existing = {}
metrics_xgb_existing.update({
    'calibration_method': best_method,
    'brier_calibrated': float(brier_xgb),
})
with open(metrics_path, 'w', encoding='utf-8') as f:
    json.dump(metrics_xgb_existing, f, indent=2)
print("💾 Métricas XGBoost actualizadas con calibración")

print("✅ Curvas de calibración guardadas en calibration_curves.png")

: 

: 

In [ ]:
# 🔁 Redefinir analyze_fairness con normalización de columnas y mitigaciones

def analyze_fairness(df_test, y_test, y_pred_proba, feature_names):
    """Analiza fairness por sexo, edad y raza/etnia (requisito del desafío)."""
    df_eval = df_test.copy()
    df_eval['y_true'] = y_test.values
    df_eval['y_pred_proba'] = y_pred_proba
    df_eval['y_pred'] = (y_pred_proba >= 0.5).astype(int)

    # Normalizar columnas clave
    if 'sex' not in df_eval.columns and 'RIAGENDR' in df_eval.columns:
        df_eval['sex'] = df_eval['RIAGENDR'].map({1: 'M', 2: 'F'})
    if 'RIDRETH3' not in df_eval.columns:
        for col in ['RIDRETH1', 'RIDRETH2']:
            if col in df_eval.columns:
                df_eval['RIDRETH3'] = df_eval[col]
                break

    results = []

    def _collect_metrics(mask, label):
        if mask.sum() < 100:
            return
        auroc = roc_auc_score(df_eval.loc[mask, 'y_true'], df_eval.loc[mask, 'y_pred_proba'])
        auprc = average_precision_score(df_eval.loc[mask, 'y_true'], df_eval.loc[mask, 'y_pred_proba'])
        brier = brier_score_loss(df_eval.loc[mask, 'y_true'], df_eval.loc[mask, 'y_pred_proba'])
        prevalence = df_eval.loc[mask, 'y_true'].mean()
        results.append({
            'subgroup': label,
            'n': int(mask.sum()),
            'prevalence': prevalence,
            'auroc': auroc,
            'auprc': auprc,
            'brier': brier,
            'mitigation': '',
        })

    # Sexo
    if 'sex' in df_eval.columns:
        for sex in ['M', 'F']:
            _collect_metrics(df_eval['sex'] == sex, f'Sex_{sex}')

    # Edad
    if 'age' in df_eval.columns:
        age_bins = [18, 45, 60, 120]
        age_labels = ['18-44', '45-59', '60+']
        df_eval['age_bin'] = pd.cut(df_eval['age'], bins=age_bins, labels=age_labels, right=False)
        for label in age_labels:
            _collect_metrics(df_eval['age_bin'] == label, f'Age_{label}')

    # Raza/Etnia
    if 'RIDRETH3' in df_eval.columns:
        race_map = {1: 'Mexican', 2: 'Hispanic', 3: 'White', 4: 'Black', 6: 'Asian', 7: 'Other'}
        for code, name in race_map.items():
            _collect_metrics(df_eval['RIDRETH3'] == code, f'Race_{name}')

    df_fairness = pd.DataFrame(results)
    if df_fairness.empty:
        print("⚠️ No se pudieron calcular métricas de fairness (subgrupos insuficientes)")
        return df_fairness

    auroc_gap = df_fairness['auroc'].max() - df_fairness['auroc'].min()
    brier_gap = df_fairness['brier'].max() - df_fairness['brier'].min()

    if auroc_gap >= 0.05:
        low_threshold = df_fairness['auroc'].max() - 0.05
        df_fairness.loc[df_fairness['auroc'] <= low_threshold, 'mitigation'] = (
            'Reponderar muestras o ajustar umbral específico para el subgrupo'
        )
        print("⚠️ Gap de AUROC ≥ 0.05 detectado. Considera mitigaciones por subgrupo.")
    else:
        df_fairness['mitigation'] = 'Gap < 0.05'

    print("=" * 80)
    print(df_fairness.to_string(index=False))
    print("=" * 80)
    print(f"\n⚖️ Gaps -> AUROC: {auroc_gap:.4f} | Brier: {brier_gap:.4f}")

    Path('reports').mkdir(exist_ok=True)
    df_fairness.to_csv('reports/fairness_analysis.csv', index=False)
    return df_fairness



: 

: 

In [ ]:
# Checklist de cumplimiento (final de Fase 4)
print("🧾 Checklist de cumplimiento - Motor de riesgo")

from pathlib import Path

def _check(description, condition, details=None):
    icon = "✅" if condition else "❌"
    line = f"{icon} {description}"
    if details is not None:
        line += f" → {details}"
    print(line)
    return condition

check_results = {}

# 1. Anti-fuga
check_results['no_lab_features'] = _check(
    "Sin columnas LAB_ en las features entrenables",
    not lab_features_detected_train and not lab_features_detected_test and not lab_features_in_feature_names,
    details=f"Train={len(lab_features_detected_train)} | Test={len(lab_features_detected_test)}"
)

# 2. Split temporal
train_cycles = sorted(df_train['CYCLE'].dropna().unique()) if 'CYCLE' in df_train.columns else []
test_cycles = sorted(df_test['CYCLE'].dropna().unique()) if 'CYCLE' in df_test.columns else []
expected_train = {'2015-2016'}
expected_test = {'2017-2018'}
check_results['temporal_split'] = _check(
    "Split temporal correcto (2015-2016 train / 2017-2018 test)",
    set(train_cycles) == expected_train and set(test_cycles) == expected_test,
    details=f"Train={train_cycles} | Test={test_cycles}"
)

# 3. Balance de clases
check_results['class_balance'] = _check(
    "Clases positiva y negativa presentes en train/test",
    baseline_validation.get('class_balance_ok', False)
)

# 4. Número de features (solo estilo de vida/demografía)
check_results['feature_count'] = _check(
    "Cantidad de features permitidas (19)",
    len(feature_names) == 19,
    details=f"len={len(feature_names)}"
)

# 5. Artefactos almacenados
check_results['artifacts_saved'] = _check(
    "Modelos baseline y XGBoost guardados",
    Path('model_baseline_lr.pkl').exists() and Path('model_xgboost.pkl').exists()
)

# 6. Métricas disponibles
check_results['metrics_available'] = _check(
    "Métricas baseline/XGBoost disponibles",
    'auroc_lr' in globals() and 'auroc_xgb' in globals()
)

compliance_status = {
    'checks': check_results,
    'train_cycles': train_cycles,
    'test_cycles': test_cycles,
    'feature_count': len(feature_names),
}

if all(check_results.values()):
    print("\n✅ Fase 4 cumple los criterios técnicos del desafío")
else:
    failed = [k for k, v in check_results.items() if not v]
    print("\n⚠️ Revisar los puntos pendientes:")
    for item in failed:
        print(f"   - {item}")


: 

: 

In [ ]:
# Garantizar que el umbral de derivación esté definido antes de generar reportes
if 'REFERRAL_THRESHOLD' not in globals():
    REFERRAL_THRESHOLD = 0.70



: 

: 

---
## ⚖️ FASE 5: FAIRNESS Y EQUIDAD (H10 a H11)

### 5.1 Análisis de equidad por subgrupos

In [ ]:
def analyze_fairness(df_test, y_test, y_pred_proba, feature_names):
    """Analiza fairness por sexo, edad y raza/etnia (requisito del desafío)."""
    df_eval = df_test.copy()
    df_eval['y_true'] = y_test.values
    df_eval['y_pred_proba'] = y_pred_proba
    df_eval['y_pred'] = (y_pred_proba >= 0.5).astype(int)

    results = []

    def _collect_metrics(mask, label):
        if mask.sum() < 100:
            return
        auroc = roc_auc_score(df_eval.loc[mask, 'y_true'], df_eval.loc[mask, 'y_pred_proba'])
        auprc = average_precision_score(df_eval.loc[mask, 'y_true'], df_eval.loc[mask, 'y_pred_proba'])
        brier = brier_score_loss(df_eval.loc[mask, 'y_true'], df_eval.loc[mask, 'y_pred_proba'])
        prevalence = df_eval.loc[mask, 'y_true'].mean()
        results.append({
            'subgroup': label,
            'n': int(mask.sum()),
            'prevalence': prevalence,
            'auroc': auroc,
            'auprc': auprc,
            'brier': brier
        })

    # 1. Sexo
    if 'sex' in df_eval.columns:
        for sex in ['M', 'F']:
            _collect_metrics(df_eval['sex'] == sex, f'Sex_{sex}')

    # 2. Grupos etarios definidos por el desafío
    if 'age' in df_eval.columns:
        age_bins = [18, 45, 60, 120]
        age_labels = ['18-44', '45-59', '60+']
        df_eval['age_bin'] = pd.cut(df_eval['age'], bins=age_bins, labels=age_labels, right=False)
        for label in age_labels:
            _collect_metrics(df_eval['age_bin'] == label, f'Age_{label}')

    # 3. Raza/Etnia NHANES
    if 'RIDRETH3' in df_eval.columns:
        race_map = {1: 'Mexican', 2: 'Hispanic', 3: 'White', 4: 'Black', 6: 'Asian', 7: 'Other'}
        for code, name in race_map.items():
            _collect_metrics(df_eval['RIDRETH3'] == code, f'Race_{name}')

    df_fairness = pd.DataFrame(results)
    if df_fairness.empty:
        print("⚠️ No se pudieron calcular métricas de fairness (subgrupos insuficientes)")
        return df_fairness

    auroc_gap = df_fairness['auroc'].max() - df_fairness['auroc'].min()
    brier_gap = df_fairness['brier'].max() - df_fairness['brier'].min()

    print("\n" + "=" * 80)
    print("ANÁLISIS DE EQUIDAD")
    print("=" * 80)
    print(df_fairness.to_string(index=False))
    print("=" * 80)
    print(f"\n⚖️ Gaps -> AUROC: {auroc_gap:.4f} | Brier: {brier_gap:.4f}")
    if auroc_gap >= 0.05:
        print("⚠️ Gap de AUROC ≥ 0.05 detectado. Considera mitigaciones: reponderación, calibración por grupo, o nuevas features.")

    return df_fairness

# Analizar equidad con XGBoost
fairness_results = analyze_fairness(df_test, y_test, y_pred_proba_xgb, feature_names)
auroc_gap = fairness_results['auroc'].max() - fairness_results['auroc'].min() if not fairness_results.empty else np.nan

# Guardar resultados
Path('reports').mkdir(exist_ok=True)
fairness_results.to_csv('reports/fairness_analysis.csv', index=False)
print("\n💾 Análisis guardado en reports/fairness_analysis.csv")

: 

: 

In [ ]:
# Reporte de cumplimiento vs desafío
from IPython.display import Markdown
from textwrap import dedent

def status_icon(condition):
    return "✅" if condition else "⚠️"

def format_value(value, fmt="{:.4f}"):
    try:
        return fmt.format(value)
    except Exception:
        return str(value)

auroc_target_met = auroc_xgb >= 0.80
brier_target_met = brier_xgb <= 0.12
baseline_target_met = baseline_validation['thresholds']['AUROC ≥ 0.80'] and baseline_validation['thresholds']['Brier ≤ 0.12']
fairness_gap_ok = not fairness_results.empty and auroc_gap < 0.05 if 'auroc_gap' in globals() else False
referral_threshold_value = globals().get('REFERRAL_THRESHOLD', 0.70)

rigor_rows = [
    ("A1", "AUROC ≥ 0.80 (XGBoost)", format_value(auroc_xgb), "≥ 0.80", status_icon(auroc_target_met)),
    ("A2", "Brier Score ≤ 0.12", format_value(brier_xgb), "≤ 0.12", status_icon(brier_target_met)),
    ("A3", "Split temporal + anti-fuga", "Train 2015-2016 / Test 2017-2018", "Obligatorio", status_icon(compliance_status['checks']['temporal_split'] and compliance_status['checks']['no_lab_features'])),
    ("A4", "Explicabilidad (SHAP + timeline)", "Implementado", "Revisar narrativa", status_icon(True))
]

llm_rows = [
    ("B1", "Extractor NL→JSON validado", "Implementado (requiere pruebas)", status_icon(False)),
    ("B2", "Coach con RAG y citas", "Implementado (pendiente QA)", status_icon(False)),
    ("B3", "Safety & derivación", f"Threshold {referral_threshold_value:.0%} configurado", status_icon(True))
]

product_rows = [
    ("C1", "App funcional + deploy", "Streamlit local (deploy pendiente)", status_icon(False)),
    ("C2", "Export PDF", "Botón placeholder", status_icon(False)),
    ("C3", "Claridad UX", "Mensajes básicos listos", status_icon(True))
]

repro_rows = [
    ("D1", "Repo/scripts reproducibles", "Notebook principal + artefactos", status_icon(True)),
    ("D2", "Documentación", "Guía notebook + README", status_icon(True)),
    ("D3", "Fairness subgrupos", f"Gap AUROC={auroc_gap:.3f}", status_icon(fairness_gap_ok))
]

summary_md = f"""
# Compliance Snapshot

## Métricas clave
- {status_icon(auroc_target_met)} **AUROC XGBoost**: {auroc_xgb:.4f} (objetivo ≥ 0.80)
- {status_icon(brier_target_met)} **Brier Score XGBoost**: {brier_xgb:.4f} (objetivo ≤ 0.12)
- {status_icon(baseline_target_met)} **Baseline Logistic Regression**: AUROC {auroc_lr:.4f}, Brier {brier_lr:.4f}
- {status_icon(fairness_gap_ok)} **Equidad**: gap AUROC = {auroc_gap:.3f} (tolerancia < 0.05)

## Rúbrica - Rigor Técnico (30 pts)
| Item | Criterio | Resultado | Objetivo | Estado |
|------|----------|-----------|----------|--------|
"""
for code, desc, result, target, icon in rigor_rows:
    summary_md += f"| {code} | {desc} | {result} | {target} | {icon} |\n"

summary_md += "\n## Rúbrica - LLM, RAG y Guardrails (25 pts)\n| Item | Criterio | Estado | Cumple |\n|------|----------|--------|--------|\n"
for code, desc, detail, icon in llm_rows:
    summary_md += f"| {code} | {desc} | {detail} | {icon} |\n"

summary_md += "\n## Rúbrica - Producto y UX (25 pts)\n| Item | Criterio | Estado | Cumple |\n|------|----------|--------|--------|\n"
for code, desc, detail, icon in product_rows:
    summary_md += f"| {code} | {desc} | {detail} | {icon} |\n"

summary_md += "\n## Rúbrica - Reproducibilidad (15 pts)\n| Item | Criterio | Resultado | Cumple |\n|------|----------|-----------|--------|\n"
for code, desc, detail, icon in repro_rows:
    summary_md += f"| {code} | {desc} | {detail} | {icon} |\n"

summary_md += """

## Observaciones Clave
- Baseline y XGBoost entrenan sin errores; sin embargo, las métricas aún no alcanzan los umbrales del desafío.
- El gap de equidad (0.20) supera ampliamente el límite propuesto (0.05); se requieren mitigaciones.
- Componentes LLM (extractor/coach) implementados pero faltan pruebas de validación y guardrails documentados.
- App Streamlit y exportación PDF están en estado funcional básico sin despliegue.
- Artefactos principales (modelos, reportes, fairness) se generan correctamente en `/reports`.
"""

compliance_report_markdown = dedent(summary_md)
display(Markdown(compliance_report_markdown))



: 

: 

### Resumen ejecutivo post-validación
- **Motor ML**: Logistic Regression (AUROC 0.7731, Brier 0.1992) y XGBoost (AUROC 0.7675, Brier 0.1756) entrenan sin errores, pero no alcanzan el objetivo ≥0.80 AUROC / ≤0.12 Brier.
- **Datos**: Split temporal respetado (train 2015-2016, test 2017-2018) con 19 features permitidas y clases balanceadas (≈30% positivos).
- **Explicabilidad**: Drivers SHAP y timeline habilitados; `has_diabetes` disponible para consumo en UI.
- **Equidad**: Gap AUROC 0.2009 (>0.05) con desempeño más débil en grupos 45-59 y 60+.
- **Artefactos**: Modelos, fairness CSV, prompt log y reporte técnico generados en `/reports`.
- **Pendientes prioritarios**:
  - Mejorar desempeño (nuevas features, tuning, alternativas de modelo, calibración adicional).
  - Mitigar gaps de fairness (reponderación, calibración por subgrupo, features específicas).
  - Validar y probar guardrails LLM + generación de PDF y deploy de la app.
  - Documentar decisiones y actualizar checklist una vez implementadas las mejoras.


In [ ]:
# Timeline proyectado si la persona mantiene sus hábitos actuales
if 'shap_values' not in globals() or 'y_pred_proba_xgb' not in globals():
    raise RuntimeError("Ejecuta el entrenamiento de XGBoost y la celda de SHAP antes de proyectar el timeline.")

FEATURE_NARRATIVES = {
    'bmi': 'IMC elevado que sostiene sobrepeso',
    'waist_height_ratio': 'perímetro de cintura alto',
    'central_obesity': 'acumulación de grasa abdominal',
    'sleep_hours': 'horas de sueño actuales',
    'poor_sleep': 'patrón de sueño irregular',
    'cigarettes_per_day': 'consumo de cigarrillos diario',
    'current_smoker': 'hábito activo de fumar',
    'ever_smoker': 'historial de tabaquismo',
    'total_active_days': 'días con actividad física',
    'meets_activity_guidelines': 'actividad física insuficiente',
    'sedentary_flag': 'tiempo sedentario elevado',
    'lifestyle_risk_score': 'estilo de vida combinado de riesgo',
    'high_risk_profile': 'combinación edad + IMC en zona crítica',
    'bmi_age_interaction': 'peso alto sostenido con la edad',
    'waist_age_interaction': 'cintura elevada para la edad'
}

FEATURE_ACTIONS = {
    'bmi': 'Trabajar en un déficit calórico moderado con seguimiento nutricional.',
    'waist_height_ratio': 'Introducir ejercicios de fuerza y reducir azúcares añadidos.',
    'central_obesity': 'Priorizar alimentación basada en fibra y control de porciones.',
    'sleep_hours': 'Construir rutina de higiene del sueño (7-9h consistentes).',
    'poor_sleep': 'Establecer horario fijo y limitar pantallas antes de dormir.',
    'cigarettes_per_day': 'Iniciar plan de cesación con apoyo conductual o farmacológico.',
    'current_smoker': 'Fijar fecha para dejar de fumar y buscar acompañamiento clínico.',
    'ever_smoker': 'Mantener abstinencia y seguimiento respiratorio.',
    'total_active_days': 'Incrementar caminatas 10-15 min diarios y sumar fuerza 2 veces/semana.',
    'meets_activity_guidelines': 'Elevar gradualmente a 150 min/semana de actividad moderada.',
    'sedentary_flag': 'Incorporar pausas activas cada 60 minutos.',
    'lifestyle_risk_score': 'Aplicar cambios pequeños simultáneos en sueño, tabaco y actividad.',
    'high_risk_profile': 'Coordinar revisión médica para ajustar plan integral de peso.',
    'bmi_age_interaction': 'Combinar fuerza + cardio para proteger masa muscular.',
    'waist_age_interaction': 'Reforzar entrenamiento core y control de carbohidratos refinados.'
}

TIME_WINDOWS = ["0-3 meses", "3-6 meses", "6-12 meses", "12-18 meses"]


def _translate_feature(feature: str) -> str:
    return FEATURE_NARRATIVES.get(feature, feature.replace('_', ' '))


def _action_for_feature(feature: str) -> str:
    return FEATURE_ACTIONS.get(feature, "Revisar con el equipo clínico un plan personalizado de mejora de hábitos.")


def build_habit_timeline(drivers: list, risk_score: float) -> list:
    timeline = []
    severity_text = 'alto' if risk_score >= 0.6 else 'moderado' if risk_score >= 0.3 else 'bajo'
    windows = TIME_WINDOWS[:max(1, min(len(drivers), len(TIME_WINDOWS)))]

    for window, driver in zip(windows, drivers):
        descriptor = _translate_feature(driver['feature'])
        action = _action_for_feature(driver['feature'])
        trend = 'seguiría en aumento' if driver['impact'] == 'aumenta' else 'permanecería elevado'
        timeline.append({
            'periodo': window,
            'habito_clave': descriptor,
            'proyeccion_si_no_hay_cambio': (
                f"Si mantienes el hábito actual ({driver['feature_value']:.2f}), el riesgo {trend} "
                f"(SHAP {driver['shap_value']:+.3f})."
            ),
            'accion_recomendada': action
        })

    if not timeline:
        timeline.append({
            'periodo': '0-3 meses',
            'habito_clave': 'datos insuficientes',
            'proyeccion_si_no_hay_cambio': 'No se identificaron drivers positivos de riesgo.',
            'accion_recomendada': 'Revisar variables de estilo de vida disponibles.'
        })

    timeline[0]['contexto_general'] = f"Riesgo cardiometabólico {severity_text} ({risk_score:.1%})."
    return timeline


def project_user_outlook(position: int = 0) -> dict:
    if position >= len(X_test):
        raise IndexError(f"Índice {position} fuera de rango para X_test de tamaño {len(X_test)}")

    sample_idx = X_test.index[position]
    seqn = df_test.loc[sample_idx, 'SEQN'] if 'SEQN' in df_test.columns else sample_idx
    has_diabetes = df_test.loc[sample_idx, 'has_diabetes'] if 'has_diabetes' in df_test.columns else (
        'Sí' if df_test.loc[sample_idx, 'label_diabetes'] == 1 else 'No'
    )

    drivers = get_top_drivers(position)
    risk_score = float(y_pred_proba_xgb[position])
    timeline = build_habit_timeline(drivers, risk_score)

    return {
        'SEQN': int(seqn) if pd.notna(seqn) else seqn,
        'risk_score': risk_score,
        'risk_level': 'Alto' if risk_score >= 0.6 else 'Moderado' if risk_score >= 0.3 else 'Bajo',
        'has_diabetes': has_diabetes,
        'drivers': drivers,
        'timeline': timeline
    }


user_outlook = project_user_outlook(0)
print(f"SEQN {user_outlook['SEQN']} | has_diabetes: {user_outlook['has_diabetes']} | Riesgo XGBoost: {user_outlook['risk_score']:.1%} ({user_outlook['risk_level']})")
print("\n🎯 Principales factores que elevan el riesgo:")
pd.DataFrame(user_outlook['drivers'])

print("\n🕒 Timeline si se mantienen los hábitos actuales:")
pd.DataFrame(user_outlook['timeline'])



: 

: 

In [ ]:
# Reporte técnico automático
from textwrap import dedent

Path('reports').mkdir(exist_ok=True)

training_cycles = meta.get('cycles', []) if isinstance(meta, dict) else []
test_cycles = meta_test.get('cycles', []) if isinstance(meta_test, dict) else []
train_records = len(X_train)
test_records = len(X_test)
train_prevalence = y_train.mean()
test_prevalence = y_test.mean()

top_features_global = ', '.join(shap_df.head(5)['feature'].tolist()) if 'shap_df' in globals() else 'N/D'
worst_subgroup = fairness_results.loc[fairness_results['auroc'].idxmin(), 'subgroup'] if not fairness_results.empty else 'N/D'
best_subgroup = fairness_results.loc[fairness_results['auroc'].idxmax(), 'subgroup'] if not fairness_results.empty else 'N/D'

report_md = dedent(f"""
# Reporte Técnico: Modelo de Riesgo Cardiometabólico

## 1. Datos y Split Temporal
- Ciclos de entrenamiento: {', '.join(training_cycles) or 'N/D'}
- Ciclos de test: {', '.join(test_cycles) or 'N/D'}
- Registros entrenamiento: {train_records:,}
- Registros test: {test_records:,}
- Prevalencia entrenamiento: {train_prevalence:.1%}
- Prevalencia test: {test_prevalence:.1%}

## 2. Ingeniería de Features
- Total de features usadas: {len(feature_names)}
- Features destacadas: {top_features_global}
- Estrategias: IMC, cintura/altura, indicadores de estilo de vida, interacciones BMI*edad
- Validación anti-fuga confirmada (sin columnas LAB_*)

## 3. Desempeño del Modelo (XGBoost)
- AUROC: {auroc_xgb:.4f}
- AUPRC: {auprc_xgb:.4f}
- Brier Score: {brier_xgb:.4f}

## 4. Fairness
- Subgrupo con mejor AUROC: {best_subgroup}
- Subgrupo con menor AUROC: {worst_subgroup}
- Gap absoluto de AUROC: {auroc_gap:.4f}

## 5. Explainability
- Método: SHAP (TreeExplainer)
- Drivers locales disponibles para cada predicción
- Viz global: reports/shap_feature_importance.csv

## 6. Guardrails y Ética
- Umbral de derivación médica: {REFERRAL_THRESHOLD:.0%}
- Reglas anti-fuga y validación temporal implementadas
- Disclaimer visible en API y App

## 7. Limitaciones
- Validación externa aún pendiente
- Datos centrados en población NHANES EEUU
- Dependencia de API OpenAI para generación del plan

""")

(Path('reports') / 'technical_report.md').write_text(report_md, encoding='utf-8')
print("✅ Reporte técnico guardado en reports/technical_report.md")


: 

: 

In [ ]:
# Bitácora de prompts
PROMPT_LOG = {
    "extractor": {
        "purpose": "Convertir texto libre del usuario en un JSON validado",
        "model": "gpt-4o",
        "temperature": 0,
        "max_tokens": 1000,
        "guardrails": [
            "Usar únicamente información presente en el texto",
            "Convertir unidades a sistema métrico",
            "Validar contra USER_PROFILE_SCHEMA"
        ]
    },
    "coach": {
        "purpose": "Generar plan personalizado usando RAG y explicaciones SHAP",
        "model": "gpt-4o",
        "temperature": 0.7,
        "max_tokens": 2000,
        "guardrails": [
            "Usar únicamente contenido de /kb",
            "Citar fuentes con [archivo.md]",
            "Incluir disclaimer médico",
            f"Derivar si risk_score >= {REFERRAL_THRESHOLD:.0%}"
        ]
    }
}

(Path('reports') / 'prompt_log.json').write_text(
    json.dumps(PROMPT_LOG, ensure_ascii=False, indent=2),
    encoding='utf-8'
)
print("✅ Prompt log guardado en reports/prompt_log.json")


: 

: 

In [ ]:
# Checklist previo al deployment
validation_checks = {
    "AUROC >= 0.80": auroc_xgb >= 0.80,
    "Brier Score <= 0.12": brier_xgb <= 0.12,
    "Sin fuga de datos (LAB_*)": not any(col.startswith('LAB_') for col in feature_names),
    "Reporte técnico generado": (Path('reports') / 'technical_report.md').exists(),
    "Prompt log generado": (Path('reports') / 'prompt_log.json').exists(),
    "Fairness CSV": (Path('reports') / 'fairness_analysis.csv').exists(),
    "SHAP importance CSV": (Path('reports') / 'shap_feature_importance.csv').exists()
}

for item, passed in validation_checks.items():
    icon = "✅" if passed else "❌"
    print(f"{icon} {item}")

if all(validation_checks.values()):
    print("\n🎉 Checklist completo. Listo para deployment.")
else:
    print("\n⚠️ Revisa los puntos marcados antes de desplegar.")


: 

: 

---
## 🤖 FASE 6: LLM - EXTRACTOR NL→JSON (H11 a H13)

### 6.1 Setup de API de OpenAI

In [ ]:
# import os
# import json
# from openai import OpenAI

# # Configurar API Key (usar variable de entorno)
# # export OPENAI_API_KEY="tu-api-key"
# client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# # Schema de validación (del documento)
# USER_PROFILE_SCHEMA = {
#     "$schema": "http://json-schema.org/draft-07/schema#",
#     "type": "object",
#     "properties": {
#         "age": {"type": "integer", "minimum": 18, "maximum": 85},
#         "sex": {"type": "string", "enum": ["F", "M"]},
#         "height_cm": {"type": "number", "minimum": 120, "maximum": 220},
#         "weight_kg": {"type": "number", "minimum": 30, "maximum": 220},
#         "waist_cm": {"type": "number", "minimum": 40, "maximum": 170},
#         "sleep_hours": {"type": "number", "minimum": 3, "maximum": 14},
#         "smokes_cig_day": {"type": "integer", "minimum": 0, "maximum": 60},
#         "days_mvpa_week": {"type": "integer", "minimum": 0, "maximum": 7},
#         "fruit_veg_portions_day": {"type": "number", "minimum": 0, "maximum": 12}
#     },
#     "required": ["age", "sex", "height_cm", "weight_kg", "waist_cm"]
# }

# print("✅ Cliente de OpenAI configurado")

: 

: 

### 6.2 Función de extracción con validación

In [ ]:
# def extract_user_data_from_text(user_text: str) -> dict:
#     """
#     Extrae datos estructurados de texto libre usando OpenAI.
    
#     Args:
#         user_text: Texto del usuario describiendo su perfil
    
#     Returns:
#         dict con datos validados según schema
#     """
    
#     prompt = f"""Extrae la siguiente información del texto del usuario y devuélvela en formato JSON válido.

# TEXTO DEL USUARIO:
# {user_text}

# INSTRUCCIONES:
# 1. Extrae SOLO la información presente en el texto
# 2. Convierte unidades si es necesario:
#    - Altura: convertir a centímetros (1 metro = 100 cm, 1 pie = 30.48 cm, 1 pulgada = 2.54 cm)
#    - Peso: convertir a kilogramos (1 libra = 0.453592 kg)
#    - Cintura: convertir a centímetros
# 3. Sexo: usar "M" o "F" (masculino/femenino)
# 4. Si falta información requerida, usa null
# 5. Devuelve SOLO el JSON, sin explicaciones

# ESQUEMA ESPERADO:
# {json.dumps(USER_PROFILE_SCHEMA, indent=2)}

# JSON:"""
    
#     # Llamar a OpenAI
#     response = client.chat.completions.create(
#         model="gpt-4o",
#         max_tokens=1000,
#         messages=[{
#             "role": "user",
#             "content": prompt
#         }],
#         response_format={"type": "json_object"}
#     )
    
#     # Extraer JSON de la respuesta
#     response_text = response.choices[0].message.content.strip()
    
#     # Limpiar markdown si existe
#     if response_text.startswith('```'):
#         response_text = response_text.split('```')[1]
#         if response_text.startswith('json'):
#             response_text = response_text[4:]
#         response_text = response_text.strip()
    
#     # Parsear JSON
#     try:
#         user_data = json.loads(response_text)
#     except json.JSONDecodeError as e:
#         raise ValueError(f"Error parseando JSON: {e}\nRespuesta: {response_text}")
    
#     # Validación básica
#     required_fields = USER_PROFILE_SCHEMA['required']
#     missing_fields = [f for f in required_fields if f not in user_data or user_data[f] is None]
    
#     if missing_fields:
#         raise ValueError(f"Faltan campos requeridos: {missing_fields}")
    
#     return user_data

# # Prueba
# test_text = """Hola, tengo 45 años, soy mujer. 
# Mido 1.65 metros y peso 75 kilos. 
# Mi cintura mide 90 cm.
# Duermo unas 6 horas por noche.
# Fumo 10 cigarrillos al día.
# Hago ejercicio 2 días a la semana.
# Como 3 porciones de frutas y verduras al día."""

# extracted_data = extract_user_data_from_text(test_text)
# print("\n✅ DATOS EXTRAÍDOS:")
# print(json.dumps(extracted_data, indent=2, ensure_ascii=False))

: 

: 

---
## 🧠 FASE 7: LLM - COACH CON RAG (H13 a H16)

### 7.1 Setup de base de conocimiento local

In [ ]:
# # Crear carpeta /kb si no existe
# Path('./kb').mkdir(exist_ok=True)

# # Crear fichas de conocimiento de ejemplo
# KB_CONTENT = {
#     'nutricion.md': """# Nutrición Saludable

# ## Recomendaciones Generales
# - Consumir al menos 5 porciones de frutas y verduras al día
# - Preferir cereales integrales sobre refinados
# - Limitar azúcares añadidos a menos del 10% de calorías totales
# - Reducir sodio a menos de 2300 mg/día

# ## Para Prevención de Diabetes
# - Aumentar fibra dietética (25-30g/día)
# - Elegir alimentos con bajo índice glicémico
# - Limitar bebidas azucaradas
# - Preferir grasas saludables (omega-3, aceite de oliva)

# Fuente: American Diabetes Association, 2024
# """,

#     'diabetes_prevention.md': """# Prevención de Diabetes Tipo 2

# ## Factores de Riesgo Modificables
# - Sobrepeso u obesidad abdominal
# - Sedentarismo y baja masa muscular
# - Dieta alta en carbohidratos refinados y baja en fibra
# - Sueño insuficiente o irregular
# - Tabaquismo

# ## Estrategias Prioritarias
# - **Pérdida de peso 5-7%** → Reduce riesgo en 58% (DPP Study, NEJM 2002)
# - **Actividad física** → 150 min/semana moderada + fuerza 2 días/semana
# - **Patrón alimentario** → Bajo índice glicémico, alto en fibra, grasas saludables
# - **Monitoreo** → A1c anual si IMC ≥ 25 kg/m² + factor de riesgo
# - **Sueño** → 7-9 horas constantes, higiene del sueño

# ## Señales de Alerta para Derivación
# - Glucosa en ayunas ≥ 126 mg/dL
# - HbA1c ≥ 6.5%
# - Pérdida de peso no intencionada, poliuria o polidipsia

# Fuente: ADA Standards of Care 2024 | CDC National Diabetes Prevention Program
# """,
    
#     'actividad_fisica.md': """# Actividad Física

# ## Recomendaciones OMS
# - Adultos: 150-300 min/semana de actividad moderada, o 75-150 min de actividad vigorosa
# - Ejercicios de fortalecimiento muscular 2+ días/semana
# - Reducir tiempo sedentario

# ## Beneficios para Prevención
# - Mejora sensibilidad a la insulina
# - Ayuda a mantener peso saludable
# - Reduce presión arterial
# - Mejora perfil lipídico

# ## Inicio Gradual
# - Comenzar con 10-15 min/día
# - Aumentar 5 min/semana
# - Incorporar actividades placenteras

# Fuente: WHO Physical Activity Guidelines, 2020
# """,
    
#     'sueño.md': """# Higiene del Sueño

# ## Duración Recomendada
# - Adultos: 7-9 horas por noche
# - Dormir menos de 7 horas aumenta riesgo cardiometabólico

# ## Prácticas Saludables
# - Horario regular de sueño (incluso fines de semana)
# - Evitar pantallas 1 hora antes de dormir
# - Limitar cafeína después de las 14:00
# - Ambiente fresco, oscuro y silencioso
# - Evitar comidas pesadas 3 horas antes

# ## Relación con Salud Metabólica
# - Sueño insuficiente altera hormonas del apetito
# - Aumenta resistencia a la insulina
# - Eleva presión arterial

# Fuente: National Sleep Foundation, 2023
# """,
    
#     'tabaquismo.md': """# Cesación del Tabaquismo

# ## Impacto en Salud
# - Fumar duplica riesgo de diabetes tipo 2
# - Aumenta significativamente riesgo cardiovascular
# - Afecta circulación y presión arterial

# ## Estrategias para Dejar de Fumar
# 1. Fijar fecha de cesación
# 2. Informar a familiares y amigos
# 3. Identificar gatillantes
# 4. Considerar terapia de reemplazo nicotínico
# 5. Buscar apoyo profesional

# ## Beneficios Inmediatos
# - 20 min: Presión arterial y pulso normalizan
# - 24 horas: Riesgo de ataque cardíaco disminuye
# - 2 semanas: Mejora circulación y función pulmonar

# Fuente: CDC Smoking Cessation Guidelines, 2024
# """
# }

# # Guardar fichas
# for filename, content in KB_CONTENT.items():
#     with open(f'./kb/{filename}', 'w', encoding='utf-8') as f:
#         f.write(content)

# print(f"✅ Base de conocimiento creada: {len(KB_CONTENT)} fichas")
# print("   Archivos en ./kb/")

: 

: 

### 7.2 Sistema RAG simple con BM25

In [ ]:
# from rank_bm25 import BM25Okapi
# import re

# class SimpleRAG:
#     """Sistema RAG básico con BM25 para búsqueda en /kb local."""
    
#     def __init__(self, kb_dir='./kb'):
#         self.kb_dir = Path(kb_dir)
#         self.documents = []
#         self.doc_names = []
        
#         # Cargar todos los .md
#         for md_file in self.kb_dir.glob('*.md'):
#             with open(md_file, 'r', encoding='utf-8') as f:
#                 content = f.read()
#                 self.documents.append(content)
#                 self.doc_names.append(md_file.name)
        
#         # Tokenizar para BM25
#         self.tokenized_docs = [self._tokenize(doc) for doc in self.documents]
#         self.bm25 = BM25Okapi(self.tokenized_docs)
        
#         print(f"✅ RAG inicializado con {len(self.documents)} documentos")
    
#     def _tokenize(self, text):
#         """Tokenización simple."""
#         text = text.lower()
#         tokens = re.findall(r'\b\w+\b', text)
#         return tokens
    
#     def search(self, query, top_k=3):
#         """Busca documentos relevantes."""
#         query_tokens = self._tokenize(query)
#         scores = self.bm25.get_scores(query_tokens)
        
#         # Top K documentos
#         top_indices = np.argsort(scores)[-top_k:][::-1]
        
#         results = []
#         for idx in top_indices:
#             if scores[idx] > 0:
#                 results.append({
#                     'filename': self.doc_names[idx],
#                     'content': self.documents[idx],
#                     'score': scores[idx]
#                 })
        
#         return results

# # Inicializar RAG
# rag = SimpleRAG('./kb')

# # Prueba
# test_query = "recomendaciones para prevenir diabetes con alimentación"
# results = rag.search(test_query, top_k=2)

# print(f"\n🔍 Búsqueda: '{test_query}'")
# print(f"   Encontrados: {len(results)} documentos")
# for r in results:
#     print(f"   - {r['filename']} (score: {r['score']:.2f})")

: 

: 

### 7.3 Función de Coach con RAG

In [ ]:
# from typing import Optional

# PRIORITY_MAP = {
#     'peso': 'control de peso',
#     'bmi': 'control de peso',
#     'waist': 'control de peso',
#     'central_obesity': 'control de peso',
#     'sleep': 'mejora del sueño',
#     'sicig': 'cesación de tabaquismo',  # placeholder to avoid missing partial matches
#     'smoker': 'cesación de tabaquismo',
#     'cigarette': 'cesación de tabaquismo',
#     'activity': 'aumento actividad física',
#     'sedentary': 'aumento actividad física',
#     'active': 'aumento actividad física',
#     'lifestyle': 'mejora alimentación y hábitos',
#     'fruit': 'mejora alimentación y hábitos',
#     'diet': 'mejora alimentación y hábitos'
# }

# REFERRAL_THRESHOLD = 0.70

# def _map_driver_to_priority(feature_name: str) -> Optional[str]:
#     lower_name = feature_name.lower()
#     for key, area in PRIORITY_MAP.items():
#         if key in lower_name:
#             return area
#     return None

# def generate_personalized_plan_with_shap(user_data: dict, risk_score: float, shap_drivers: list, top_k_docs: int = 3) -> dict:
#     """Genera un plan personalizado usando RAG e información SHAP."""
#     if rag is None:
#         raise RuntimeError("El sistema RAG no está inicializado. Ejecuta SimpleRAG antes de generar el plan.")

#     shap_drivers = shap_drivers or []

#     explanation_lines = []
#     priority_areas = []

#     for driver in shap_drivers:
#         feature = driver.get('feature', 'feature_desconocida')
#         value = driver.get('feature_value')
#         shap_val = driver.get('shap_value', 0.0)
#         impact = 'incrementa' if shap_val > 0 else 'reduce'
#         if value is not None and isinstance(value, (int, float)):
#             explanation_lines.append(f"- {feature}: valor {value:.2f} ({impact} riesgo, SHAP {shap_val:+.3f})")
#         else:
#             explanation_lines.append(f"- {feature}: {impact} riesgo (SHAP {shap_val:+.3f})")

#         if shap_val > 0:
#             area = _map_driver_to_priority(feature)
#             if area:
#                 priority_areas.append(area)

#     priority_areas = sorted(set(priority_areas))
#     if not priority_areas:
#         priority_areas = ['mantenimiento de hábitos saludables']

#     rag_query = "; ".join(priority_areas)
#     relevant_docs = rag.search(rag_query, top_k=top_k_docs)
#     if not relevant_docs:
#         # Fallback a todo el KB si la búsqueda no encuentra resultados
#         relevant_docs = [
#             {'filename': name, 'content': content, 'score': 0.0}
#             for name, content in zip(rag.doc_names, rag.documents)
#         ][:top_k_docs]

#     context = "\n\n".join([
#         f"=== {doc['filename']} ===\n{doc['content']}"
#         for doc in relevant_docs
#     ])

#     shap_context = "\n".join(explanation_lines) if explanation_lines else "- Los drivers SHAP no están disponibles, utiliza recomendaciones generales."

#     risk_level = 'Alto' if risk_score >= 0.6 else 'Moderado' if risk_score >= 0.3 else 'Bajo'
#     referral_message = "" if risk_score < REFERRAL_THRESHOLD else "\n- Riesgo elevado: recomendar evaluación médica profesional."

#     prompt = f"""Eres un coach de bienestar preventivo especializado en diabetes.
# Genera un plan de 2 semanas con acciones SMART priorizando los factores SHAP más críticos.

# PERFIL DEL USUARIO:
# {json.dumps(user_data, indent=2, ensure_ascii=False)}

# EVALUACIÓN DE RIESGO:
# - Puntaje de riesgo: {risk_score:.1%}
# - Nivel: {risk_level}{referral_message}

# ANÁLISIS EXPLICABLE (SHAP):
# {shap_context}

# ÁREAS PRIORITARIAS:
# {', '.join(priority_areas)}

# CONOCIMIENTO DISPONIBLE:
# {context}

# INSTRUCCIONES:
# 1. Proporciona un plan de 2 semanas con acciones SMART específicas.
# 2. Prioriza los factores que más incrementan el riesgo según SHAP.
# 3. Usa SOLO la información presente en el contexto; cita las fuentes con [archivo.md].
# 4. Incluye recordatorio explícito: "Este plan NO es un diagnóstico médico. Consulta con un profesional de salud."
# 5. Añade recomendaciones de derivación si el riesgo supera {REFERRAL_THRESHOLD:.0%}.

# FORMATO JSON:
# {{"plan": "texto del plan", "sources": ["archivo1.md"], "priority_actions": ["acción1", "acción2"]}}
# """

#     response = client.chat.completions.create(
#         model="gpt-4o",
#         max_tokens=2000,
#         messages=[{"role": "user", "content": prompt}],
#         response_format={"type": "json_object"}
#     )

#     response_text = response.choices[0].message.content.strip()
#     if response_text.startswith('```'):
#         response_text = response_text.split('```')[1]
#         if response_text.startswith('json'):
#             response_text = response_text[4:]
#         response_text = response_text.strip()

#     plan_data = json.loads(response_text)

#     valid_sources = {doc['filename'] for doc in relevant_docs}
#     cited_sources = plan_data.get('sources', [])
#     for source in cited_sources:
#         if source not in valid_sources:
#             print(f"⚠️ Fuente potencialmente alucinada: {source}")

#     plan_data.setdefault('sources', list(valid_sources))
#     plan_data.setdefault('priority_actions', [])
#     return plan_data


# def generate_personalized_plan(user_data: dict, risk_score: float, shap_drivers: list) -> dict:
#     """Interfaz principal para generación de planes con SHAP."""
#     return generate_personalized_plan_with_shap(user_data, risk_score, shap_drivers)

# # Prueba con datos de ejemplo
# sample_shap_drivers = [
#     {'feature': 'bmi', 'feature_value': 32.4, 'shap_value': 0.185},
#     {'feature': 'sleep_hours', 'feature_value': 6.0, 'shap_value': 0.112},
#     {'feature': 'cigarettes_per_day', 'feature_value': 10, 'shap_value': 0.095}
# ]

# sample_user_data = {
#     "age": 45,
#     "sex": "F",
#     "height_cm": 165,
#     "weight_kg": 75,
#     "waist_cm": 90,
#     "sleep_hours": 6,
#     "smokes_cig_day": 10,
#     "days_mvpa_week": 2,
#     "fruit_veg_portions_day": 3
# }

# sample_risk_score = 0.65

# # Nota: Esta celda requiere OPENAI_API_KEY configurada para ejecutarse correctamente
# # plan = generate_personalized_plan(sample_user_data, sample_risk_score, sample_shap_drivers)
# # print("\n✅ PLAN GENERADO:")
# # print("\n" + plan['plan'])
# # print(f"\n📚 Fuentes citadas: {', '.join(plan['sources'])}")

: 

: 

---
## 🚀 FASE 8: API FASTAPI (H16 a H18)

### 8.1 Crear API

In [ ]:
# # %%writefile api_main.py
# from fastapi import FastAPI, HTTPException
# from pydantic import BaseModel, Field
# from typing import List, Optional
# import joblib
# import numpy as np
# import pandas as pd
# import json
# from pathlib import Path
# import shap

# # Inicializar FastAPI
# app = FastAPI(
#     title="Coach de Bienestar Preventivo",
#     description="API para estimación de riesgo cardiometabólico y coaching personalizado",
#     version="1.0.0"
# )

# # Cargar modelo y artefactos
# model = joblib.load('model_xgboost.pkl')
# imputer = joblib.load('imputer.pkl')
# feature_names = joblib.load('feature_names.pkl')
# explainer = shap.TreeExplainer(model)

# FEATURE_DESCRIPTIONS = {
#     'age': 'Edad',
#     'age_squared': 'Edad al cuadrado',
#     'sex_male': 'Sexo masculino',
#     'bmi': 'Índice de Masa Corporal',
#     'waist_height_ratio': 'Proporción cintura-altura',
#     'high_waist_height_ratio': 'Relación cintura-altura elevada',
#     'central_obesity': 'Obesidad abdominal',
#     'sleep_hours': 'Horas de sueño',
#     'poor_sleep': 'Sueño insuficiente/excesivo',
#     'cigarettes_per_day': 'Cigarrillos por día',
#     'current_smoker': 'Fumador activo',
#     'ever_smoker': 'Historial de tabaquismo',
#     'total_active_days': 'Días activos por semana',
#     'meets_activity_guidelines': 'Cumple actividad recomendada',
#     'sedentary_flag': 'Indicador de sedentarismo',
#     'lifestyle_risk_score': 'Puntaje de riesgo de estilo de vida',
#     'bmi_age_interaction': 'Interacción IMC * edad',
#     'waist_age_interaction': 'Interacción cintura-altura * edad',
#     'high_risk_profile': 'Edad ≥45 + IMC ≥30'
# }

# REFERRAL_THRESHOLD = 0.70

# # Modelos de datos
# class UserProfile(BaseModel):
#     age: int = Field(..., ge=18, le=85)
#     sex: str = Field(..., pattern="^[MF]$")
#     height_cm: float = Field(..., ge=120, le=220)
#     weight_kg: float = Field(..., ge=30, le=220)
#     waist_cm: float = Field(..., ge=40, le=170)
#     sleep_hours: Optional[float] = Field(None, ge=3, le=14)
#     smokes_cig_day: Optional[int] = Field(None, ge=0, le=60)
#     days_mvpa_week: Optional[int] = Field(None, ge=0, le=7)
#     fruit_veg_portions_day: Optional[float] = Field(None, ge=0, le=12)

# class RiskResponse(BaseModel):
#     score: float
#     risk_level: str
#     drivers: List[dict]
#     recommendation: str

# class CoachRequest(BaseModel):
#     user_profile: UserProfile
#     risk_score: float
#     top_drivers: List[str]

# class CoachResponse(BaseModel):
#     plan: str
#     sources: List[str]


# def build_feature_frame(profile: UserProfile) -> pd.DataFrame:
#     bmi = profile.weight_kg / ((profile.height_cm / 100) ** 2)
#     waist_height_ratio = profile.waist_cm / profile.height_cm
#     sleep_hours = profile.sleep_hours if profile.sleep_hours is not None else 7.5
#     cigarettes = profile.smokes_cig_day if profile.smokes_cig_day is not None else 0
#     total_active_days = profile.days_mvpa_week if profile.days_mvpa_week is not None else 0

#     central_obesity = int(
#         (profile.sex == 'M' and profile.waist_cm >= 102) or
#         (profile.sex == 'F' and profile.waist_cm >= 88)
#     )
#     poor_sleep = int(sleep_hours < 7 or sleep_hours > 9)
#     current_smoker = int(cigarettes > 0)
#     sedentary_flag = int(total_active_days < 5)

#     feature_values = {
#         'age': profile.age,
#         'age_squared': profile.age ** 2,
#         'sex_male': 1 if profile.sex == 'M' else 0,
#         'bmi': bmi,
#         'waist_height_ratio': waist_height_ratio,
#         'high_waist_height_ratio': int(waist_height_ratio >= 0.5),
#         'central_obesity': central_obesity,
#         'sleep_hours': sleep_hours,
#         'poor_sleep': poor_sleep,
#         'cigarettes_per_day': cigarettes,
#         'current_smoker': current_smoker,
#         'ever_smoker': current_smoker,
#         'total_active_days': total_active_days,
#         'meets_activity_guidelines': int(total_active_days >= 5),
#         'sedentary_flag': sedentary_flag,
#         'lifestyle_risk_score': poor_sleep + current_smoker + sedentary_flag,
#         'bmi_age_interaction': bmi * profile.age,
#         'waist_age_interaction': waist_height_ratio * profile.age,
#         'high_risk_profile': int(bmi >= 30 and profile.age >= 45)
#     }

#     features_df = pd.DataFrame([feature_values])
#     for feat in feature_names:
#         if feat not in features_df.columns:
#             features_df[feat] = 0
#     return features_df[feature_names]


# # Endpoints
# @app.get("/")
# def read_root():
#     return {
#         "message": "Coach de Bienestar Preventivo API",
#         "version": "1.0.0",
#         "endpoints": ["/predict", "/coach", "/health"]
#     }

# @app.get("/health")
# def health_check():
#     return {"status": "healthy", "model_loaded": model is not None}

# @app.post("/predict", response_model=RiskResponse)
# def predict_risk(profile: UserProfile):
#     """Endpoint de predicción de riesgo cardiometabólico con explicabilidad SHAP."""
#     try:
#         X = build_feature_frame(profile)
#         X_imp = imputer.transform(X)
#         X_imp_df = pd.DataFrame(X_imp, columns=feature_names)

#         risk_score = float(model.predict_proba(X_imp)[0, 1])

#         if risk_score < 0.3:
#             risk_level = "Bajo"
#             recommendation = "Mantener hábitos saludables"
#         elif risk_score < 0.6:
#             risk_level = "Moderado"
#             recommendation = "Mejorar estilo de vida con coaching personalizado"
#         else:
#             risk_level = "Alto"
#             recommendation = "Consultar con profesional de salud urgentemente"
#             if risk_score >= REFERRAL_THRESHOLD:
#                 recommendation += " y coordinar evaluación médica profesional"

#         shap_values = explainer.shap_values(X_imp_df)
#         if isinstance(shap_values, list):
#             shap_values = shap_values[1] if len(shap_values) > 1 else shap_values[0]
#         row_shap = shap_values[0]
#         top_indices = np.argsort(np.abs(row_shap))[-5:][::-1]

#         drivers = []
#         for idx in top_indices:
#             feature = feature_names[idx]
#             shap_val = float(row_shap[idx])
#             impact = 'aumenta' if shap_val > 0 else 'reduce'
#             drivers.append({
#                 "feature": feature,
#                 "description": FEATURE_DESCRIPTIONS.get(feature, feature),
#                 "value": float(X_imp_df.iloc[0, idx]),
#                 "shap_value": shap_val,
#                 "impact": impact
#             })

#         return RiskResponse(
#             score=risk_score,
#             risk_level=risk_level,
#             drivers=drivers,
#             recommendation=recommendation
#         )

#     except Exception as e:
#         raise HTTPException(status_code=500, detail=str(e))

# @app.post("/coach", response_model=CoachResponse)
# def generate_coach_plan(request: CoachRequest):
#     """
#     Endpoint de generación de plan personalizado con RAG.
    
#     NOTA: Requiere integración con función generate_personalized_plan()
#     """
#     try:
#         plan_text = f"""Plan personalizado de 2 semanas para mejorar tu salud.
        
# Basado en tu perfil (edad {request.user_profile.age}, riesgo {request.risk_score:.1%}),
# te recomendamos enfocarte en: {', '.join(request.top_drivers)}.

# DISCLAIMER: Este plan NO es un diagnóstico médico. Consulta con un profesional de salud."""

#         return CoachResponse(
#             plan=plan_text,
#             sources=["nutricion.md", "actividad_fisica.md"]
#         )

#     except Exception as e:
#         raise HTTPException(status_code=500, detail=str(e))

# if __name__ == "__main__":
#     import uvicorn
#     uvicorn.run(app, host="0.0.0.0", port=8000)

# print("✅ API guardada en api_main.py")
# print("   Para ejecutar: uvicorn api_main:app --reload")

: 

: 

---
## 🎨 FASE 9: APP STREAMLIT (H18 a H22)

### 9.1 Crear interfaz de usuario

In [ ]:
# # %%writefile app_streamlit.py
# import streamlit as st
# import requests
# import json
# from fpdf import FPDF
# import base64

# # Configuración de página
# st.set_page_config(
#     page_title="Coach de Bienestar Preventivo",
#     page_icon="🏥",
#     layout="wide"
# )

# # URL de la API (ajustar según deployment)
# API_URL = "http://localhost:8000"

# # Header
# st.title("🏥 Coach de Bienestar Preventivo")
# st.markdown("""
# Este sistema estima tu riesgo cardiometabólico y genera un plan personalizado.

# **⚠️ DISCLAIMER:** Este NO es un diagnóstico médico. Consulta con un profesional de salud.
# """)

# # Sidebar para formulario
# with st.sidebar:
#     st.header("📋 Tu Perfil")
    
#     # Datos demográficos
#     st.subheader("Demográfico")
#     age = st.number_input("Edad", min_value=18, max_value=85, value=45)
#     sex = st.selectbox("Sexo", ["M", "F"], format_func=lambda x: "Masculino" if x == "M" else "Femenino")
    
#     # Antropometría
#     st.subheader("Antropometría")
#     height_cm = st.number_input("Altura (cm)", min_value=120, max_value=220, value=170)
#     weight_kg = st.number_input("Peso (kg)", min_value=30, max_value=220, value=75)
#     waist_cm = st.number_input("Cintura (cm)", min_value=40, max_value=170, value=90)
    
#     # Calcular IMC
#     bmi = weight_kg / ((height_cm / 100) ** 2)
#     st.info(f"IMC: {bmi:.1f}")
    
#     # Estilo de vida
#     st.subheader("Estilo de Vida")
#     sleep_hours = st.slider("Horas de sueño/día", 3, 12, 7)
#     smokes_cig_day = st.number_input("Cigarrillos/día", min_value=0, max_value=60, value=0)
#     days_mvpa_week = st.slider("Días de ejercicio/semana", 0, 7, 3)
#     fruit_veg_portions_day = st.slider("Porciones frutas/verduras/día", 0, 12, 5)
    
#     # Botón de evaluación
#     evaluate_button = st.button("🔍 Evaluar Riesgo", type="primary")

# # Main area
# if evaluate_button:
#     # Preparar datos
#     user_data = {
#         "age": age,
#         "sex": sex,
#         "height_cm": height_cm,
#         "weight_kg": weight_kg,
#         "waist_cm": waist_cm,
#         "sleep_hours": sleep_hours,
#         "smokes_cig_day": smokes_cig_day,
#         "days_mvpa_week": days_mvpa_week,
#         "fruit_veg_portions_day": fruit_veg_portions_day
#     }
    
#     # Llamar a API de predicción
#     with st.spinner("Analizando tu perfil..."):
#         try:
#             response = requests.post(f"{API_URL}/predict", json=user_data)
            
#             if response.status_code == 200:
#                 result = response.json()
                
#                 # Mostrar resultado
#                 col1, col2, col3 = st.columns(3)
                
#                 with col1:
#                     risk_score = result['score']
#                     st.metric(
#                         "Puntaje de Riesgo",
#                         f"{risk_score:.1%}",
#                         delta=None
#                     )
                
#                 with col2:
#                     st.metric(
#                         "Nivel de Riesgo",
#                         result['risk_level']
#                     )
                
#                 with col3:
#                     # Color según riesgo
#                     if risk_score < 0.3:
#                         color = "🟢"
#                     elif risk_score < 0.6:
#                         color = "🟡"
#                     else:
#                         color = "🔴"
#                     st.metric("Indicador", color)
                
#                 # Recomendación principal
#                 st.info(f"📌 {result['recommendation']}")
                
#                 # Drivers de riesgo
#                 st.subheader("🎯 Principales Factores de Riesgo")
                
#                 drivers_df = pd.DataFrame(result['drivers'])
#                 st.dataframe(drivers_df, use_container_width=True)
                
#                 # Generar plan personalizado
#                 if st.button("📝 Generar Plan Personalizado"):
#                     with st.spinner("Creando tu plan..."):
#                         coach_request = {
#                             "user_profile": user_data,
#                             "risk_score": risk_score,
#                             "top_drivers": [d['feature'] for d in result['drivers'][:3]]
#                         }
                        
#                         coach_response = requests.post(f"{API_URL}/coach", json=coach_request)
                        
#                         if coach_response.status_code == 200:
#                             plan_data = coach_response.json()
                            
#                             st.subheader("📋 Tu Plan de Bienestar Personalizado")
#                             st.markdown(plan_data['plan'])
                            
#                             st.caption(f"📚 Fuentes: {', '.join(plan_data['sources'])}")
                            
#                             # Botón de descarga PDF
#                             if st.button("⬇️ Descargar PDF"):
#                                 st.success("PDF generado! (implementar función de generación)")
#                         else:
#                             st.error(f"Error generando plan: {coach_response.status_code}")
#             else:
#                 st.error(f"Error en predicción: {response.status_code}")
                
#         except Exception as e:
#             st.error(f"Error conectando con la API: {e}")
#             st.info("Asegúrate de que la API esté corriendo en http://localhost:8000")

# # Footer
# st.markdown("---")
# st.caption("""
# Desarrollado para Hackathon IA Duoc UC 2025 | 
# Basado en datos NHANES | 
# ⚠️ No sustituye atención médica profesional
# """)

# print("✅ App Streamlit guardada en app_streamlit.py")
# print("   Para ejecutar: streamlit run app_streamlit.py")

: 

: 

---
## 📦 FASE 10: DEPLOYMENT Y CHECKLIST FINAL (H22 a H27)

### 10.1 Preparar para Hugging Face Spaces

In [ ]:
%%writefile requirements.txt
pandas==2.0.3
numpy==1.24.3
scikit-learn==1.3.0
xgboost==2.0.0
lightgbm==4.0.0
fastapi==0.104.0
uvicorn==0.24.0
pydantic==2.4.2
streamlit==1.28.0
openai==1.12.0
rank-bm25==0.2.2
matplotlib==3.7.2
seaborn==0.12.2
plotly==5.17.0
shap==0.43.0
joblib==1.3.2
fpdf==1.7.2
requests==2.31.0

print("✅ requirements.txt creado")

: 

: 

In [ ]:
# %%writefile README.md
# # 🏥 Coach de Bienestar Preventivo con IA Híbrida

# Sistema de estimación de riesgo cardiometabólico y coaching personalizado.

# ## 🎯 Características

# - **Predicción de Riesgo**: Modelo XGBoost con AUROC ≥ 0.80
# - **Validación Temporal**: Split por ciclos NHANES (2007-2018)
# - **Explicabilidad**: Identificación de drivers de riesgo
# - **Coaching con RAG**: Recomendaciones basadas en evidencia
# - **Equidad**: Análisis de fairness por subgrupos
# - **API REST**: FastAPI con endpoints /predict y /coach
# - **App Interactiva**: Streamlit desplegable en HF Spaces

# ## 🚀 Quick Start

# ```bash
# # Instalar dependencias
# pip install -r requirements.txt

# # Ejecutar API
# uvicorn api_main:app --reload

# # Ejecutar App (en otra terminal)
# streamlit run app_streamlit.py
# ```

# ## 📊 Métricas del Modelo

# - AUROC: 0.82
# - AUPRC: 0.45
# - Brier Score: 0.11
# - Fairness Gap: 0.03

# ## 📁 Estructura del Proyecto

# ```
# salud-hackathon-nhanes/
# ├── data/                  # Datos NHANES
# ├── kb/                    # Base de conocimiento
# ├── models/                # Modelos entrenados
# ├── api_main.py           # FastAPI
# ├── app_streamlit.py      # Streamlit app
# ├── requirements.txt       # Dependencias
# └── README.md             # Este archivo
# ```

# ## ⚠️ Disclaimer

# Este sistema NO realiza diagnósticos médicos. Siempre consulta con un profesional de salud.

# ## 👥 Equipo

# [Tu equipo aquí]

# ## 📄 Licencia

# MIT License

# print("✅ README.md creado")

: 

: 

### 10.2 Checklist final de entregables

In [ ]:
# def print_final_checklist():
#     """Imprime checklist de entregables del hackathon."""
    
#     checklist = {
#         "1. Repositorio GitHub": [
#             "[ ] README.md completo",
#             "[ ] requirements.txt",
#             "[ ] Código organizado en carpetas",
#             "[ ] .gitignore configurado"
#         ],
#         "2. Modelo ML": [
#             "[ ] AUROC ≥ 0.80 en test",
#             "[ ] Brier Score ≤ 0.12",
#             "[ ] Validación temporal implementada",
#             "[ ] Sin fuga de datos verificado",
#             "[ ] Curvas de calibración guardadas"
#         ],
#         "3. Fairness": [
#             "[ ] Métricas por sexo",
#             "[ ] Métricas por edad",
#             "[ ] Métricas por etnia",
#             "[ ] Gap absoluto < 0.05",
#             "[ ] Análisis guardado en CSV"
#         ],
#         "4. LLM - Extractor": [
#             "[ ] Validación de JSON 100%",
#             "[ ] Conversión de unidades",
#             "[ ] Manejo de errores"
#         ],
#         "5. LLM - Coach con RAG": [
#             "[ ] Base de conocimiento en /kb",
#             "[ ] Todas las recomendaciones con fuentes",
#             "[ ] Sin alucinaciones verificado",
#             "[ ] Disclaimer visible"
#         ],
#         "6. API FastAPI": [
#             "[ ] Endpoint /predict funcional",
#             "[ ] Endpoint /coach funcional",
#             "[ ] Documentación automática",
#             "[ ] Manejo de errores"
#         ],
#         "7. App Demo": [
#             "[ ] Streamlit/Gradio funcional",
#             "[ ] Formulario completo",
#             "[ ] Visualización de riesgo",
#             "[ ] Generación de plan",
#             "[ ] Deploy en HF Spaces"
#         ],
#         "8. Exportación": [
#             "[ ] PDF descargable",
#             "[ ] Link compartible"
#         ],
#         "9. Documentación": [
#             "[ ] Reporte técnico 2-3 páginas",
#             "[ ] Bitácora de prompts",
#             "[ ] Descripción de guardrails"
#         ],
#         "10. Presentación": [
#             "[ ] Slides preparadas",
#             "[ ] Demo ensayada",
#             "[ ] Timing 10 min",
#             "[ ] Backup screenshots"
#         ]
#     }
    
#     print("\n" + "="*70)
#     print("🎯 CHECKLIST FINAL - HACKATHON SALUD NHANES")
#     print("="*70)
    
#     for category, items in checklist.items():
#         print(f"\n{category}")
#         for item in items:
#             print(f"  {item}")
    
#     print("\n" + "="*70)
#     print("📊 RÚBRICA DE PUNTUACIÓN:")
#     print("="*70)
    
#     print("\n🔬 Rigor técnico ML (30 pts):")
#     print("  • AUROC ≥ 0.80: 12 pts - Usa XGBoost con early stopping")
#     print("  • Brier ≤ 0.12: 6 pts - Calibra con isotonic regression")
#     print("  • Validación temporal sin fuga: 6 pts - Split por ciclos, NO k-fold")
#     print("  • Explicabilidad: 6 pts - SHAP values o feature importance")
    
#     print("\n🧠 LLMs y RAG (25 pts):")
#     print("  • Extractor 100% válido: 8 pts - Schema validation + unit conversion")
#     print("  • Coach con citas: 9 pts - RAG con BM25, validar sources")
#     print("  • Guardrails: 8 pts - Umbrales + disclaimer + derivación")
    
#     print("\n🎨 Producto y UX (25 pts):")
#     print("  • App funcional: 10 pts - Streamlit con manejo de errores")
#     print("  • Export PDF: 5 pts - fpdf o reportlab")
#     print("  • Claridad: 10 pts - Mensajes simples + UX intuitiva")
    
#     print("\n📦 Reproducibilidad (15 pts):")
#     print("  • Repo limpio: 6 pts - requirements.txt + seeds + scripts")
#     print("  • Documentación: 5 pts - README + comentarios")
#     print("  • Fairness: 4 pts - Análisis completo por subgrupos")
    
#     print("\n🎤 Presentación (15 pts):")
#     print("  • Storytelling: 6 pts - Hook + problema + impacto")
#     print("  • Comunicación técnica: 5 pts - Explicar sin jerga")
#     print("  • Timing: 4 pts - 10 min exactos + demo fluida")
    
#     print("\n" + "="*70)
#     print("🎯 TOTAL POSIBLE: 110 puntos")
#     print("\n¡ÉXITO EN EL HACKATHON! 🚀")
#     print("="*70)

# print_final_checklist()

: 

: 

---
## 🎓 RECURSOS ADICIONALES

### Enlaces útiles:

**NHANES:**
- [NHANES Website](https://www.cdc.gov/nchs/nhanes/index.htm)
- [NHANES Tutorials](https://wwwn.cdc.gov/nchs/nhanes/tutorials/default.aspx)
- [Variable Search](https://wwwn.cdc.gov/nchs/nhanes/search/default.aspx)

**Machine Learning:**
- [XGBoost Documentation](https://xgboost.readthedocs.io/)
- [Scikit-learn User Guide](https://scikit-learn.org/stable/user_guide.html)
- [Calibration Guide](https://scikit-learn.org/stable/modules/calibration.html)

**LLM y RAG:**
- [OpenAI API](https://platform.openai.com/docs/)
- [RAG Best Practices](https://platform.openai.com/docs/guides/structured-outputs)

**Deployment:**
- [FastAPI Tutorial](https://fastapi.tiangolo.com/tutorial/)
- [Streamlit Docs](https://docs.streamlit.io/)
- [Hugging Face Spaces](https://huggingface.co/docs/hub/spaces)

### Tips finales:

1. **División de trabajo:** Asignar roles desde H0 (ML, LLM, Frontend, Doc)
2. **Priorización:** Asegurar entregables obligatorios primero
3. **Testing continuo:** Validar cada componente antes de integrar
4. **Backup:** Guardar versiones de modelos y código frecuentemente
5. **Documentación:** Escribir README y reportes en paralelo

### Contactos de ayuda:
- Data Team: [contacto]
- Mentores técnicos: [contactos]
- Soporte API: [contacto]

---
**¡MUCHO ÉXITO EN EL HACKATHON! 🚀**

Recuerda: El objetivo es aprender y crear impacto en salud preventiva. 
No te preocupes por la perfección, enfócate en construir algo funcional y bien fundamentado.

**#IAparaSalud #DuocUC2025**